<a href="https://colab.research.google.com/github/sneha-cornell/windborne/blob/main/windborne_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

url = "https://a.windbornesystems.com/treasure/00.json"

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    print(f"Successfully accessed {url}")
    print("First 500 characters of the response content:")
    print(response.text[:500])

except requests.exceptions.HTTPError as e:
    print(f"HTTP error accessing {url}: {e}")
except requests.exceptions.RequestException as e:
    print(f"Error accessing {url}: {e}")

Successfully accessed https://a.windbornesystems.com/treasure/00.json
First 500 characters of the response content:
[
    [
        70.78929088655421,
        37.27219027444302,
        2.70364806446168
    ],
    [
        47.25266228831717,
        -62.017660533384024,
        16.381159064339887
    ],
    [
        -17.851458743005086,
        -46.9200645059151,
        18.524807923676363
    ],
    [
        -8.862796351672573,
        142.93586931064723,
        12.292537951987399
    ],
    [
        -16.140941516021545,
        74.20295762937728,
        9.099438437551088
    ],
    [
        40.895374


In [ ]:
import requests
import json

def find_list_of_candidates(data):
    """Recursively searches for a list that potentially contains balloon data (dicts or lists)."""
    if isinstance(data, list):
        # Check if the list contains dicts or other lists
        if any(isinstance(item, (dict, list)) for item in data):
             return data
        # If it's a list of primitive types, and not just one, it might be our data
        elif len(data) > 1 and all(isinstance(item, (int, float)) for item in data):
             return data
        else:
             # If it's a list but not dicts or lists, check items within the list (e.g., list of numbers)
            for item in data:
                found = find_list_of_candidates(item)
                if found:
                    return found

    elif isinstance(data, dict):
        # If it's a dictionary, check its values
        for key in ['balloons', 'data', 'positions', 'flights']: # Prioritize known keys
            if key in data and isinstance(data[key], list):
                 # Check if the list under this key contains dicts or other lists
                 if any(isinstance(item, (dict, list)) for item in data[key]):
                    return data[key]
        # If not found in known keys, check all values
        for value in data.values():
            found = find_list_of_candidates(value)
            if found:
                return found
    return None


def extract_balloon_data(raw_data, hour_index):
    extracted = []
    candidates = find_list_of_candidates(raw_data)

    if not candidates:
        print(f"Could not find a list of candidates in data from hour {hour_index}. Raw data type: {type(raw_data)}")
        return extracted

    for i, entry in enumerate(candidates):
        try:
            lat = None
            lon = None
            balloon_id = None
            timestamp = f"{hour_index:02d}H_ago" # Default timestamp

            if isinstance(entry, dict):
                lat = entry.get('lat') or entry.get('latitude')
                lon = entry.get('lon') or entry.get('lng') or entry.get('longitude')
                balloon_id = entry.get('id') or entry.get('balloon_id') or entry.get('name')
                timestamp = entry.get('timestamp') or entry.get('time') or entry.get('ts')

            elif isinstance(entry, list) and len(entry) >= 2:
                 # Assume format is [lat, lon, ...]
                lat = entry[0]
                lon = entry[1]
                # Attempt to create a simple ID for list-based entries
                balloon_id = f"balloon_{hour_index:02d}_{i}"

            if lat is not None and lon is not None:
                # Attempt to convert lat and lon to float, skip if not possible
                try:
                    lat = float(lat)
                    lon = float(lon)
                except (ValueError, TypeError):
                    print(f"Skipping entry at index {i} from hour {hour_index} due to non-numeric lat/lon: lat={lat}, lon={lon}")
                    continue

                # Add timestamp if it was found in the dict, otherwise use default
                final_timestamp = timestamp if isinstance(entry, dict) and (entry.get('timestamp') or entry.get('time') or entry.get('ts')) else f"{hour_index:02d}H_ago"

                extracted.append({
                    'id': balloon_id,
                    'lat': lat,
                    'lon': lon,
                    'timestamp': final_timestamp,
                    'raw': entry # Store the raw entry for debugging if needed
                })
            else:
                # Log entries that were candidates but didn't yield lat/lon
                if isinstance(entry, (dict, list)):
                     print(f"Skipping candidate entry at index {i} from hour {hour_index} due to missing lat/lon: {entry}")
                else:
                     print(f"Skipping non-dictionary/list entry at index {i} from hour {hour_index}: {type(entry)}")


        except (TypeError, AttributeError, IndexError, Exception) as e:
            print(f"Error processing entry at index {i} from hour {hour_index}: {entry}. Error: {e}")
            continue # Continue to the next entry even if one fails

    return extracted


def fetch_and_extract_all():
    all_balloon_data = []
    for hour in range(24):
        url = f"https://a.windbornesystems.com/treasure/{hour:02d}.json"
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
            try:
                # Attempt to load JSON, but catch specific errors
                data = response.json()
                extracted = extract_balloon_data(data, hour)
                all_balloon_data.extend(extracted)
            except json.JSONDecodeError as e:
                print(f"Malformed JSON at {url}: {e}, skipping.")
            except Exception as e:
                 print(f"Error processing JSON data from {url}: {e}")
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404:
                print(f"File not found at {url}, skipping.")
            else:
                print(f"HTTP error fetching {url}: {e}")
        except Exception as e:
            print(f"Error fetching {url}: {e}")
    return all_balloon_data

def main():
    all_data = fetch_and_extract_all()
    print(f"Extracted {len(all_data)} valid balloon position entries.")
    with open('windborne_balloon_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)
    print("Saved results to windborne_balloon_data.json")

if __name__ == "__main__":
    main()

Error processing JSON data from https://a.windbornesystems.com/treasure/01.json: Expecting value: line 753 column 9 (char 14295)
Error processing JSON data from https://a.windbornesystems.com/treasure/02.json: Extra data: line 6 column 6 (char 94)
File not found at https://a.windbornesystems.com/treasure/04.json, skipping.
Error processing JSON data from https://a.windbornesystems.com/treasure/05.json: Expecting value: line 2298 column 9 (char 43703)
File not found at https://a.windbornesystems.com/treasure/06.json, skipping.
Error processing JSON data from https://a.windbornesystems.com/treasure/08.json: Expecting value: line 978 column 9 (char 18596)
Error processing JSON data from https://a.windbornesystems.com/treasure/09.json: Extra data: line 5 column 6 (char 93)
Error processing JSON data from https://a.windbornesystems.com/treasure/10.json: Expecting value: line 988 column 9 (char 18692)
Error processing JSON data from https://a.windbornesystems.com/treasure/11.json: Extra data

In [ ]:
import requests
import os
import json # Import json for JSONDecodeError
from google.colab import userdata # Import userdata

def get_weather_data(lat, lon):
    """
    Fetches wind speed and direction from OpenWeatherMap API for a given location.

    Args:
        lat (float): Latitude of the location.
        lon (float) : Longitude of the location.

    Returns:
        tuple: A tuple containing (wind_speed, wind_direction) in m/s and degrees,
               or (None, None) if the API call is unsuccessful or data is missing.
    """
    # Attempt to get the API key using userdata.get()
    api_key = userdata.get('OPENWEATHERMAP_API_KEY')
    if not api_key:
        print("OPENWEATHERMAP_API_KEY secret not found using userdata.get(). Please ensure it's set in Colab secrets.")
        return None, None

    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}lat={lat}&lon={lon}&appid={api_key}&units=metric"

    try:
        response = requests.get(complete_url, timeout=10)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        if "wind" in data:
            wind_speed = data["wind"].get("speed")
            wind_direction = data["wind"].get("deg")
            return wind_speed, wind_direction
        else:
            print(f"Wind data not found for lat={lat}, lon={lon}")
            return None, None

    except requests.exceptions.HTTPError as e:
        print(f"HTTP error fetching weather data for lat={lat}, lon={lon}: {e}")
    except requests.exceptions.ConnectionError as e:
        print(f"Connection error fetching weather data for lat={lat}, lon={lon}: {e}")
    except requests.exceptions.Timeout as e:
        print(f"Timeout error fetching weather data for lat={lat}, lon={lon}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data for lat={lat}, lon={lon}: {e}")
    except json.JSONDecodeError:
        print(f"Malformed JSON response for lat={lat}, lon={lon}")
    except Exception as e:
        print(f"An unexpected error occurred fetching weather data for lat={lat}, lon={lon}: {e}")

    return None, None

In [ ]:
# Example usage to print some data from OpenWeatherMap
# Test coordinates (e.g., New York City)
test_lat = 40.7128
test_lon = -74.0060

# Add print statement to confirm API key is fetched
import os
# Removed the print statement that uses os.environ.get() as it was causing confusion.
# The get_weather_data function now uses userdata.get() internally.


print(f"Attempting to fetch weather data for lat={test_lat}, lon={test_lon}...")
wind_speed, wind_direction = get_weather_data(test_lat, test_lon)

if wind_speed is not None and wind_direction is not None:
    print(f"Successfully fetched weather data:")
    print(f"  Wind Speed: {wind_speed} m/s")
    print(f"  Wind Direction: {wind_direction} degrees")
else:
    print(f"Failed to fetch weather data for lat={test_lat}, lon={test_lon}. Check the error messages above for details.")

Attempting to fetch weather data for lat=40.7128, lon=-74.006...
Successfully fetched weather data:
  Wind Speed: 6.17 m/s
  Wind Direction: 210 degrees


## Data Combination and Visualization

### Subtask:
Combine the balloon data with weather data and create a visualization.

**Reasoning**:
Combine the balloon data (assuming it is loaded or can be loaded from `windborne_balloon_data.json`) with weather data by iterating through the balloon positions and fetching weather data for each. Then, create a basic visualization of this combined data.

In [ ]:
import pandas as pd
import json
import os
import time
import plotly.express as px
import plotly.graph_objects as go

# --- Configuration for smaller batch ---
BATCH_SIZE = 100 # Define the number of balloon entries to process for visualization testing

# --- Step 1: Load Balloon Data ---
# Ensure this file exists by running the balloon data acquisition cell (I38C1hkKEBQZ) first
try:
    with open('windborne_balloon_data.json', 'r', encoding='utf-8') as f:
        all_balloon_data = json.load(f)
except FileNotFoundError:
    print("Error: windborne_balloon_data.json not found. Please run the balloon data acquisition step (cell I38C1hkKEBQZ) first.")
    all_balloon_data = []
except json.JSONDecodeError:
    print("Error: Could not decode windborne_balloon_data.json. The file might be corrupted.")
    all_balloon_data = []
except Exception as e:
    print(f"An unexpected error occurred while loading windborne_balloon_data.json: {e}")
    all_balloon_data = []

print(f"Loaded {len(all_balloon_data)} total balloon position entries.")

# --- Select a smaller batch for combination and visualization ---
balloon_data_batch = all_balloon_data[:BATCH_SIZE]
print(f"Selected a batch of {len(balloon_data_batch)} entries for combination and visualization.")


# --- Step 2: Data Combination (Fetch Weather Data and Combine) ---
# This assumes the get_weather_data function (cell 00963023) is defined and works.
# Ensure get_weather_data function is defined
if 'get_weather_data' not in locals():
     print("Error: get_weather_data function is not defined. Please ensure the external data acquisition step (cell 00963023) was run successfully.")
     combined_data = balloon_data_batch # Keep original data if weather fetching is not possible
else:
    combined_data = []
    weather_fetch_count = 0
    successful_fetches = 0
    API_CALL_DELAY = 0.1 # seconds delay between weather API calls

    if balloon_data_batch: # Proceed only if the batch is not empty
        print("Starting data combination with weather data for the batch...")
        for i, entry in enumerate(balloon_data_batch):
            lat = entry.get('lat')
            lon = entry.get('lon')

            # Ensure lat and lon are floats before passing to get_weather_data
            try:
                lat = float(lat) if lat is not None else None
                lon = float(lon) if lon is not None else None
            except (ValueError, TypeError):
                print(f"Skipping weather fetch for entry {i} in batch due to invalid lat/lon type: lat={lat}, lon={lon}")
                lat = None
                lon = None


            if lat is not None and lon is not None:
                # Fetch weather data for the balloon's location
                # This will use userdata.get() internally based on our previous modification
                wind_speed, wind_direction = get_weather_data(lat, lon)
                weather_fetch_count += 1

                # Add weather data to the balloon entry
                if wind_speed is not None and wind_direction is not None:
                    entry['wind_speed'] = wind_speed
                    entry['wind_direction'] = wind_direction
                    successful_fetches += 1
                else:
                    # Handle cases where weather data could not be fetched for this specific point
                    # print(f"Could not fetch weather data for entry {i} in batch at lat={lat}, lon={lon}") # Optional: keep print for debugging
                    entry['wind_speed'] = None
                    entry['wind_direction'] = None

                combined_data.append(entry)

                # Add a delay after each API call to respect rate limits
                time.sleep(API_CALL_DELAY)

            else:
                # If lat/lon is missing or invalid, we cannot fetch weather, so we add the entry without weather data.
                combined_data.append(entry)

        print(f"Attempted to fetch weather data for {weather_fetch_count} entries in the batch.")
        print(f"Successfully fetched weather data for {successful_fetches} entries in the batch.")
        print(f"Combined data contains {len(combined_data)} entries.")

        # --- Save the combined data for the Flask app ---
        if combined_data:
            try:
                with open('windborne_balloon_weather_combined.json', 'w', encoding='utf-8') as f:
                    json.dump(combined_data, f, ensure_ascii=False, indent=2)
                print("Saved combined results (batch) to windborne_balloon_weather_combined.json")
            except Exception as e:
                print(f"Error saving combined data: {e}")
        else:
             print("No combined data to save.")

    else:
        print("Skipping data combination and saving as the balloon data batch is empty.")


# --- Step 3: Prepare Data for Visualization ---
# This part remains for immediate visualization testing after combination in this cell.
# The Flask app will load the saved JSON separately.
if combined_data:
    # Convert the list of dictionaries to a pandas DataFrame
    df_combined_viz = pd.DataFrame(combined_data) # Use a different variable name to avoid confusion with df_combined used for ML later

    # Ensure latitude and longitude columns are numeric
    df_combined_viz['lat'] = pd.to_numeric(df_combined_viz['lat'], errors='coerce')
    df_combined_viz['lon'] = pd.to_numeric(df_combined_viz['lon'], errors='coerce')
    df_combined_viz['wind_speed'] = pd.to_numeric(df_combined_viz['wind_speed'], errors='coerce')
    df_combined_viz['wind_direction'] = pd.to_numeric(df_combined_viz['wind_direction'], errors='coerce')

    # Drop rows where lat or lon are not valid numbers after coercion
    df_combined_viz.dropna(subset=['lat', 'lon'], inplace=True)

    print(f"DataFrame created with {len(df_combined_viz)} entries after cleaning for visualization.")

    # --- Step 4: Create a Visualization ---
    # We can create a scatter plot on a map showing balloon positions,
    # perhaps colored by wind speed or direction if available.

    if not df_combined_viz.empty:
        print("Generating visualization...")
        # Basic scatter plot on a map
        fig = px.scatter_mapbox(df_combined_viz,
                                lat="lat",
                                lon="lon",
                                color="wind_speed", # Color points by wind speed
                                size="wind_speed",   # Size points by wind speed (optional)
                                hover_name="id",     # Show balloon ID on hover
                                hover_data={"lat": True, "lon": True, "wind_speed": True, "wind_direction": True, "timestamp": True},
                                color_continuous_scale=px.colors.cyclical.IceFire,
                                size_max=15,
                                zoom=1,
                                height=500)

        fig.update_layout(mapbox_style="open-street-map") # Use OpenStreetMap base map
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.update_layout(title_text=f"Balloon Positions ({len(df_combined_viz)} entries) with Wind Speed (Batch)") # Add a title
        fig.show()

        print("Visualization generated.")

    else:
        print("No valid data points to visualize after cleaning for visualization.")

else:
    print("Cannot visualize data as no balloon data was loaded or combined.")

Loaded 3000 total balloon position entries.
Selected a batch of 100 entries for combination and visualization.
Starting data combination with weather data for the batch...
Attempted to fetch weather data for 100 entries in the batch.
Successfully fetched weather data for 100 entries in the batch.
Combined data contains 100 entries.
Saved combined results (batch) to windborne_balloon_weather_combined.json
DataFrame created with 100 entries after cleaning for visualization.
Generating visualization...


Visualization generated.


In [ ]:
import requests
import json

url = "https://a.windbornesystems.com/treasure/00.json"

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    raw_content = response.text
    print(f"Successfully accessed {url}")
    print(f"Type of raw content: {type(raw_content)}")
    print("First 500 characters of raw content:")
    print(raw_content[:500])

    # Attempt to parse as JSON
    try:
        data = json.loads(raw_content)
        print(f"\nSuccessfully parsed JSON from {url}")
        print(f"Type of parsed data: {type(data)}")
        print("Structure of parsed data (first few items/keys):")
        if isinstance(data, list):
            print(data[:5])
        elif isinstance(data, dict):
            # Print keys and types of values for the first few keys
            for i, (key, value) in enumerate(data.items()):
                if i >= 5: break
                print(f"  Key: {key}, Type of value: {type(value)}")
        else:
            print(data)

    except json.JSONDecodeError as e:
        print(f"\nCould not parse JSON from {url}: {e}")

except requests.exceptions.HTTPError as e:
    print(f"HTTP error accessing {url}: {e}")
except requests.exceptions.RequestException as e:
    print(f"Error accessing {url}: {e}")

Successfully accessed https://a.windbornesystems.com/treasure/00.json
Type of raw content: <class 'str'>
First 500 characters of raw content:
[
    [
        70.78929088655421,
        37.27219027444302,
        2.70364806446168
    ],
    [
        47.25266228831717,
        -62.017660533384024,
        16.381159064339887
    ],
    [
        -17.851458743005086,
        -46.9200645059151,
        18.524807923676363
    ],
    [
        -8.862796351672573,
        142.93586931064723,
        12.292537951987399
    ],
    [
        -16.140941516021545,
        74.20295762937728,
        9.099438437551088
    ],
    [
        40.895374

Successfully parsed JSON from https://a.windbornesystems.com/treasure/00.json
Type of parsed data: <class 'list'>
Structure of parsed data (first few items/keys):
[[70.78929088655421, 37.27219027444302, 2.70364806446168], [47.25266228831717, -62.017660533384024, 16.381159064339887], [-17.851458743005086, -46.9200645059151, 18.524807923676363], [-8.862796

# Task
Combine the extracted Windborne balloon data and OpenWeatherMap data, and then apply a Machine Learning technique to gain insights or make predictions, such as predicting wind fields. Visualize the results and ensure the process can dynamically update with new data.

## Prepare data for ml

### Subtask:
Format the combined balloon and weather data into a structure suitable for Machine Learning. This might involve creating features from the existing data, such as changes in balloon position, time differences, and corresponding wind conditions.


**Reasoning**:
The combined data is already in `df_combined`. I need to calculate the change in latitude and longitude, and the time difference between consecutive points for each balloon. This requires sorting the data by balloon ID and timestamp, then using pandas' `diff()` method. I will also calculate the balloon's speed.



In [ ]:
# Ensure data is sorted by balloon ID and timestamp for correct difference calculation
df_combined = df_combined.sort_values(by=['id', 'timestamp'])

# Calculate the difference in latitude and longitude for consecutive points of the same balloon
df_combined['delta_lat'] = df_combined.groupby('id')['lat'].diff()
df_combined['delta_lon'] = df_combined.groupby('id')['lon'].diff()

# Calculate the time difference. Assuming 'timestamp' is in a format that can be converted to datetime.
# If 'timestamp' is '00H_ago', '01H_ago', etc., we need to convert it to a numerical representation first.
# For this batch, the timestamp is '00H_ago', so time difference will be NaN.
# If the full dataset had different timestamps, we would convert them.
# Let's assume for now that if the full dataset was used, timestamps would allow for diff calculation.
# For this batch with only '00H_ago', time_diff will be NaN.
# If timestamps were actual datetime objects or sequential numbers:
# df_combined['timestamp_dt'] = pd.to_datetime(df_combined['timestamp']) # Example conversion
# df_combined['time_diff'] = df_combined.groupby('id')['timestamp_dt'].diff().dt.total_seconds() # Example time diff in seconds

# Since the current batch only has '00H_ago', we'll just add a placeholder or skip time_diff calculation for now.
# If the task requires predicting wind fields based on movement, time difference is crucial.
# Let's add a placeholder and acknowledge this limitation with the current batch.
df_combined['time_diff'] = None # Placeholder for time difference

# Calculate the distance moved (simplified Euclidean distance for small changes)
# This is a proxy for balloon speed if time_diff were available.
# For this batch, delta_lat and delta_lon are calculated, but time_diff is not.
# We can calculate the displacement, but not speed.
df_combined['displacement'] = (df_combined['delta_lat']**2 + df_combined['delta_lon']**2)**0.5

# Handle missing values created by diff() (the first entry for each balloon)
# These rows cannot be used to predict the wind based on previous movement.
df_combined.dropna(subset=['delta_lat', 'delta_lon'], inplace=True)

# Display the first few rows with the new features
display(df_combined.head())

,id,lat,lon,timestamp,raw,wind_speed,wind_direction,delta_lat,delta_lon,time_diff,displacement


**Reasoning**:
The previous command resulted in an empty DataFrame because the batch size was small and likely contained only one entry per balloon ID, making `diff()` produce NaN values which were then dropped. To address this and create features suitable for ML, I need to ensure the DataFrame contains data with consecutive points for at least some balloon IDs. Given the previous code loaded 3000 entries into `all_balloon_data`, I will use a larger batch size to increase the chance of having multiple entries per balloon ID after sorting. I will then re-apply the feature engineering steps: calculate `delta_lat`, `delta_lon`, `time_diff`, and `displacement`, and handle the resulting NaNs.



In [ ]:
# Increase the batch size to include more data points
BATCH_SIZE = 500 # Use a larger batch size

# --- Step 1: Load Balloon Data ---
# Ensure this file exists by running the balloon data acquisition cell (I38C1hkKEBQZ) first
try:
    with open('windborne_balloon_data.json', 'r', encoding='utf-8') as f:
        all_balloon_data = json.load(f)
except FileNotFoundError:
    print("Error: windborne_balloon_data.json not found. Please run the balloon data acquisition step (cell I38C1hkKEBQZ) first.")
    all_balloon_data = []
except json.JSONDecodeError:
    print("Error: Could not decode windborne_balloon_data.json. The file might be corrupted.")
    all_balloon_data = []
except Exception as e:
    print(f"An unexpected error occurred while loading windborne_balloon_data.json: {e}")
    all_balloon_data = []

print(f"Loaded {len(all_balloon_data)} total balloon position entries.")

# --- Select a larger batch for combination and visualization ---
balloon_data_batch = all_balloon_data[:BATCH_SIZE]
print(f"Selected a batch of {len(balloon_data_batch)} entries for combination and visualization.")


# --- Step 2: Data Combination (Fetch Weather Data and Combine) ---
# This assumes the get_weather_data function (cell 00963023) is defined and works.
# Ensure get_weather_data function is defined
if 'get_weather_data' not in locals():
     print("Error: get_weather_data function is not defined. Please ensure the external data acquisition step (cell 00963023) was run successfully.")
     combined_data = balloon_data_batch # Keep original data if weather fetching is not possible
else:
    combined_data = []
    weather_fetch_count = 0
    successful_fetches = 0
    API_CALL_DELAY = 0.1 # seconds delay between weather API calls

    if balloon_data_batch: # Proceed only if the batch is not empty
        print("Starting data combination with weather data for the batch...")
        for i, entry in enumerate(balloon_data_batch):
            lat = entry.get('lat')
            lon = entry.get('lon')

            # Ensure lat and lon are floats before passing to get_weather_data
            try:
                lat = float(lat) if lat is not None else None
                lon = float(lon) if lon is not None else None
            except (ValueError, TypeError):
                print(f"Skipping weather fetch for entry {i} in batch due to invalid lat/lon type: lat={lat}, lon={lon}")
                lat = None
                lon = None


            if lat is not None and lon is not None:
                # Fetch weather data for the balloon's location
                # This will use userdata.get() internally based on our previous modification
                wind_speed, wind_direction = get_weather_data(lat, lon)
                weather_fetch_count += 1

                # Add weather data to the balloon entry
                if wind_speed is not None and wind_direction is not None:
                    entry['wind_speed'] = wind_speed
                    entry['wind_direction'] = wind_direction
                    successful_fetches += 1
                else:
                    # Handle cases where weather data could not be fetched for this specific point
                    # print(f"Could not fetch weather data for entry {i} in batch at lat={lat}, lon={lon}") # Optional: keep print for debugging
                    entry['wind_speed'] = None
                    entry['wind_direction'] = None

                combined_data.append(entry)

                # Add a delay after each API call to respect rate limits
                time.sleep(API_CALL_DELAY)

            else:
                # If lat/lon is missing or invalid, we cannot fetch weather, so we add the entry without weather data.
                combined_data.append(entry)

        print(f"Attempted to fetch weather data for {weather_fetch_count} entries in the batch.")
        print(f"Successfully fetched weather data for {successful_fetches} entries in the batch.")
        print(f"Combined data contains {len(combined_data)} entries.")
    else:
        print("Skipping data combination as the balloon data batch is empty.")

# --- Step 3: Prepare Data for Visualization ---
if combined_data:
    # Convert the list of dictionaries to a pandas DataFrame
    df_combined = pd.DataFrame(combined_data)

    # Ensure latitude and longitude columns are numeric
    df_combined['lat'] = pd.to_numeric(df_combined['lat'], errors='coerce')
    df_combined['lon'] = pd.to_numeric(df_combined['lon'], errors='coerce')
    df_combined['wind_speed'] = pd.to_numeric(df_combined['wind_speed'], errors='coerce')
    df_combined['wind_direction'] = pd.to_numeric(df_combined['wind_direction'], errors='coerce')

    # Drop rows where lat or lon are not valid numbers after coercion
    df_combined.dropna(subset=['lat', 'lon'], inplace=True)

    print(f"DataFrame created with {len(df_combined)} entries after cleaning.")

    # --- Step 4: Feature Engineering for ML ---

    # Ensure data is sorted by balloon ID and timestamp for correct difference calculation
    # Assuming 'timestamp' is a string like '00H_ago', '01H_ago', etc. Convert to a sortable format.
    # We can extract the hour number for sorting.
    df_combined['timestamp_hour'] = df_combined['timestamp'].str.extract(r'(\d+)H_ago').astype(float)
    df_combined = df_combined.sort_values(by=['id', 'timestamp_hour'])

    # Calculate the difference in latitude and longitude for consecutive points of the same balloon
    df_combined['delta_lat'] = df_combined.groupby('id')['lat'].diff()
    df_combined['delta_lon'] = df_combined.groupby('id')['lon'].diff()

    # Calculate the time difference in hours (assuming timestamps like '00H_ago', '01H_ago' are 1 hour apart)
    # If the timestamp format is consistent 'XXH_ago', the difference in 'timestamp_hour' is the time difference in hours.
    df_combined['time_diff_hours'] = df_combined.groupby('id')['timestamp_hour'].diff()

    # Calculate the displacement (Euclidean distance)
    df_combined['displacement'] = (df_combined['delta_lat']**2 + df_combined['delta_lon']**2)**0.5

    # Calculate approximate speed (displacement / time_diff)
    # Avoid division by zero or NaN time_diff
    df_combined['speed_approx'] = df_combined['displacement'] / df_combined['time_diff_hours']

    # Handle missing values created by diff() (the first entry for each balloon)
    # These rows cannot be used to predict wind based on previous movement.
    df_combined.dropna(subset=['delta_lat', 'delta_lon', 'time_diff_hours', 'displacement', 'speed_approx'], inplace=True)


    print(f"DataFrame after feature engineering and dropping NaNs: {len(df_combined)} entries.")

    # Display the first few rows with the new features
    display(df_combined.head())

else:
    print("Cannot perform feature engineering as no balloon data was loaded or combined.")

Loaded 3000 total balloon position entries.
Selected a batch of 500 entries for combination and visualization.
Starting data combination with weather data for the batch...
Attempted to fetch weather data for 500 entries in the batch.
Successfully fetched weather data for 500 entries in the batch.
Combined data contains 500 entries.
DataFrame created with 500 entries after cleaning.
DataFrame after feature engineering and dropping NaNs: 0 entries.


,id,lat,lon,timestamp,raw,wind_speed,wind_direction,timestamp_hour,delta_lat,delta_lon,time_diff_hours,displacement,speed_approx


**Reasoning**:
The DataFrame is still empty after feature engineering and dropping NaNs. This indicates that even with a batch size of 500, there are no consecutive data points for the same balloon ID with differing timestamps within this batch that also have weather data. To create a DataFrame suitable for ML, I need to use the *entire* dataset of 3000 balloon entries loaded from `windborne_balloon_data.json` before fetching weather data for them, and then perform the feature engineering steps. This maximizes the chance of finding consecutive points for balloons across different timestamps.



In [ ]:
# --- Step 1: Load ALL Balloon Data ---
# Ensure this file exists by running the balloon data acquisition cell (I38C1hkKEBQZ) first
try:
    with open('windborne_balloon_data.json', 'r', encoding='utf-8') as f:
        all_balloon_data = json.load(f)
except FileNotFoundError:
    print("Error: windborne_balloon_data.json not found. Please run the balloon data acquisition step (cell I38C1hkKEBQZ) first.")
    all_balloon_data = []
except json.JSONDecodeError:
    print("Error: Could not decode windborne_balloon_data.json. The file might be corrupted.")
    all_balloon_data = []
except Exception as e:
    print(f"An unexpected error occurred while loading windborne_balloon_data.json: {e}")
    all_balloon_data = []

print(f"Loaded {len(all_balloon_data)} total balloon position entries.")

# --- Step 2: Data Combination (Fetch Weather Data for ALL Data and Combine) ---
# This assumes the get_weather_data function (cell 00963023) is defined and works.
# Ensure get_weather_data function is defined
if 'get_weather_data' not in locals():
     print("Error: get_weather_data function is not defined. Please ensure the external data acquisition step (cell 00963023) was run successfully.")
     combined_data = all_balloon_data # Keep original data if weather fetching is not possible
else:
    combined_data = []
    weather_fetch_count = 0
    successful_fetches = 0
    API_CALL_DELAY = 0.1 # seconds delay between weather API calls

    if all_balloon_data: # Proceed only if the data is not empty
        print("Starting data combination with weather data for the entire dataset...")
        for i, entry in enumerate(all_balloon_data):
            lat = entry.get('lat')
            lon = entry.get('lon')

            # Ensure lat and lon are floats before passing to get_weather_data
            try:
                lat = float(lat) if lat is not None else None
                lon = float(lon) if lon is not None else None
            except (ValueError, TypeError):
                print(f"Skipping weather fetch for entry {i} due to invalid lat/lon type: lat={lat}, lon={lon}")
                lat = None
                lon = None


            if lat is not None and lon is not None:
                # Fetch weather data for the balloon's location
                # This will use userdata.get() internally based on our previous modification
                wind_speed, wind_direction = get_weather_data(lat, lon)
                weather_fetch_count += 1

                # Add weather data to the balloon entry
                if wind_speed is not None and wind_direction is not None:
                    entry['wind_speed'] = wind_speed
                    entry['wind_direction'] = wind_direction
                    successful_fetches += 1
                else:
                    # Handle cases where weather data could not be fetched for this specific point
                    # print(f"Could not fetch weather data for entry {i} at lat={lat}, lon={lon}") # Optional: keep print for debugging
                    entry['wind_speed'] = None
                    entry['wind_direction'] = None

                combined_data.append(entry)

                # Add a delay after each API call to respect rate limits
                time.sleep(API_CALL_DELAY)

            else:
                # If lat/lon is missing or invalid, we cannot fetch weather, so we add the entry without weather data.
                combined_data.append(entry)

        print(f"Attempted to fetch weather data for {weather_fetch_count} entries in the dataset.")
        print(f"Successfully fetched weather data for {successful_fetches} entries in the dataset.")
        print(f"Combined data contains {len(combined_data)} entries.")
    else:
        print("Skipping data combination as the balloon data is empty.")

# --- Step 3: Prepare Data for Feature Engineering ---
if combined_data:
    # Convert the list of dictionaries to a pandas DataFrame
    df_combined = pd.DataFrame(combined_data)

    # Ensure latitude and longitude columns are numeric
    df_combined['lat'] = pd.to_numeric(df_combined['lat'], errors='coerce')
    df_combined['lon'] = pd.to_numeric(df_combined['lon'], errors='coerce')
    df_combined['wind_speed'] = pd.to_numeric(df_combined['wind_speed'], errors='coerce')
    df_combined['wind_direction'] = pd.to_numeric(df_combined['wind_direction'], errors='coerce')

    # Drop rows where lat or lon are not valid numbers after coercion
    df_combined.dropna(subset=['lat', 'lon'], inplace=True)

    print(f"DataFrame created with {len(df_combined)} entries after cleaning.")

    # --- Step 4: Feature Engineering for ML ---

    # Ensure data is sorted by balloon ID and timestamp for correct difference calculation
    # Assuming 'timestamp' is a string like '00H_ago', '01H_ago', etc. Convert to a sortable format.
    # We can extract the hour number for sorting.
    df_combined['timestamp_hour'] = df_combined['timestamp'].str.extract(r'(\d+)H_ago').astype(float)
    df_combined = df_combined.sort_values(by=['id', 'timestamp_hour'])

    # Calculate the difference in latitude and longitude for consecutive points of the same balloon
    df_combined['delta_lat'] = df_combined.groupby('id')['lat'].diff()
    df_combined['delta_lon'] = df_combined.groupby('id')['lon'].diff()

    # Calculate the time difference in hours (assuming timestamps like '00H_ago', '01H_ago' are 1 hour apart)
    # If the timestamp format is consistent 'XXH_ago', the difference in 'timestamp_hour' is the time difference in hours.
    df_combined['time_diff_hours'] = df_combined.groupby('id')['timestamp_hour'].diff()

    # Calculate the displacement (Euclidean distance)
    df_combined['displacement'] = (df_combined['delta_lat']**2 + df_combined['delta_lon']**2)**0.5

    # Calculate approximate speed (displacement / time_diff)
    # Avoid division by zero or NaN time_diff
    df_combined['speed_approx'] = df_combined['displacement'] / df_combined['time_diff_hours']

    # Handle missing values created by diff() (the first entry for each balloon)
    # These rows cannot be used to predict wind based on previous movement.
    df_combined.dropna(subset=['delta_lat', 'delta_lon', 'time_diff_hours', 'displacement', 'speed_approx'], inplace=True)


    print(f"DataFrame after feature engineering and dropping NaNs: {len(df_combined)} entries.")

    # Display the first few rows with the new features
    display(df_combined.head())

else:
    print("Cannot perform feature engineering as no balloon data was loaded or combined.")

Loaded 3000 total balloon position entries.
Starting data combination with weather data for the entire dataset...
Attempted to fetch weather data for 3000 entries in the dataset.
Successfully fetched weather data for 3000 entries in the dataset.
Combined data contains 3000 entries.
DataFrame created with 3000 entries after cleaning.
DataFrame after feature engineering and dropping NaNs: 0 entries.


,id,lat,lon,timestamp,raw,wind_speed,wind_direction,timestamp_hour,delta_lat,delta_lon,time_diff_hours,displacement,speed_approx


## Define ml problem and model

### Subtask:
Define the specific Machine Learning problem based on the available data and choose an appropriate ML model or technique, considering the limitation that sequential balloon data is not available for movement-based wind prediction.


**Reasoning**:
Define the ML problem and choose a suitable model based on the available features and limitations.



In [ ]:
# Re-evaluate available features in df_combined
# Based on the current state, the features that are available after combining data are:
# 'id', 'lat', 'lon', 'timestamp', 'raw', 'wind_speed', 'wind_direction', 'timestamp_hour'
# Features like 'delta_lat', 'delta_lon', 'time_diff_hours', 'displacement', 'speed_approx' are NOT available
# because there are no sequential data points for individual balloons.

# Available input features: 'lat', 'lon', 'timestamp_hour' (representing the hour of the day)
# Potential target variables: 'wind_speed', 'wind_direction'

# --- Define the Alternative Machine Learning Problem ---
# Given the limitation of no sequential balloon movement data, we cannot predict wind fields based on
# how balloons move over time.
# An alternative problem is to predict the wind speed and wind direction at a given geographical location
# (latitude, longitude) and time (represented by the hour of the day) based on the observed weather data points.
# This is a supervised learning regression problem where we use 'lat', 'lon', and 'timestamp_hour' as features
# to predict 'wind_speed' and 'wind_direction'.

# Problem Definition:
# Predict wind_speed and wind_direction given latitude, longitude, and hour of the day.

# --- Choose Appropriate Machine Learning Model(s) ---
# Since this is a regression problem with numerical input features and numerical target variables,
# several standard regression models are suitable.
# Considering the potential complexity of wind patterns, non-linear models might perform better than simple linear models.

# Suitable models:
# 1. Linear Regression: Simple baseline model.
# 2. Decision Tree Regressor: Can capture non-linear relationships.
# 3. Random Forest Regressor: Ensemble method, generally robust and good at capturing complex interactions.
# 4. Gradient Boosting Regressor (e.g., scikit-learn's GradientBoostingRegressor or XGBoost, LightGBM): Can provide high accuracy but might require more tuning.

# Given the exploratory nature and the potential for non-linear spatial and temporal patterns in wind data,
# Random Forest Regressor is a good starting choice as it's relatively easy to use and often provides
# good performance without extensive hyperparameter tuning. Decision Tree Regressor is also a reasonable choice for
# its interpretability, although it might be prone to overfitting.

# Selected Model(s): Random Forest Regressor and potentially Decision Tree Regressor as a simpler alternative.

# --- Document the Problem and Model Choices (in a markdown cell later) ---
# The documentation will explain:
# - The ML problem: Predicting wind speed and direction at a location/time.
# - The input features: latitude, longitude, hour of the day.
# - The target variables: wind speed, wind direction.
# - The reason for this problem definition (data limitations).
# - The chosen models (Random Forest Regressor, Decision Tree Regressor) and why they are suitable.

**Reasoning**:
Document the defined ML problem and selected models in a markdown cell as planned.



In [ ]:
### Machine Learning Problem Definition and Model Selection

# **Revised Machine Learning Problem:**

# Based on the available data, which provides discrete balloon positions and corresponding wind speed and direction measurements at those specific points in time, but *lacks sequential data for individual balloons*, the initial goal of predicting wind fields based on balloon *movement* is not feasible.

# Instead, we define an alternative supervised learning problem:

# **Predicting wind speed and wind direction at a given geographical location (latitude, longitude) and time (represented by the hour of the day).**

# *   **Input Features (X):**
#     *   `lat`: Latitude of the location.
#     *   `lon`: Longitude of the location.
#     *   `timestamp_hour`: The hour of the day when the observation was made.
# *   **Target Variables (y):**
#     *   `wind_speed`: The observed wind speed at the location and time.
#     *   `wind_direction`: The observed wind direction at the location and time.

# This is a **regression** problem, where we are trying to predict continuous numerical values (wind speed and direction).

# **Rationale for Problem Redefinition:**

# The constraint of having only single, non-sequential observations per balloon ID prevents the calculation of movement-based features (`delta_lat`, `delta_lon`, `time_diff_hours`, `displacement`, `speed_approx`). Therefore, we leverage the available static features (location and time) and the directly observed wind conditions to build a model that can estimate wind parameters at unobserved locations or times within the data's spatial and temporal range.

# **Selected Machine Learning Models:**

# Given the nature of the problem (regression) and the desire to capture potentially complex, non-linear relationships between location/time and wind conditions, the following models are suitable candidates:

# 1.  **Random Forest Regressor:** An ensemble learning method that builds multiple decision trees and merges their predictions. It is generally robust, less prone to overfitting than individual decision trees, and can capture non-linear interactions between features. This is a strong candidate for its balance of performance and ease of use.
# 2.  **Decision Tree Regressor (Alternative/Simpler):** A simpler tree-based model. While potentially prone to overfitting on its own, it provides interpretability regarding which features (latitude, longitude, hour) are most influential in determining wind conditions at specific points. It could serve as a baseline or for initial exploration.

# We will primarily focus on the **Random Forest Regressor** for its expected better performance in capturing the spatial and temporal variations in wind data.


## Split data

### Subtask:
Split the prepared dataset (`df_combined`) into training and testing sets.


**Reasoning**:
Import the necessary function and split the data into training and testing sets using the identified features and target variables, then print the shapes.



In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variables (y)
# Features: 'lat', 'lon', 'timestamp_hour'
# Targets: 'wind_speed', 'wind_direction'

# Ensure df_combined is available and not empty before attempting to split
if 'df_combined' in locals() and not df_combined.empty:
    # Select the features and target variables that are available and relevant to the ML problem
    X = df_combined[['lat', 'lon', 'timestamp_hour']]
    y = df_combined[['wind_speed', 'wind_direction']]

    # Drop any rows with NaN values in the selected features or targets before splitting
    # This is crucial because train_test_split cannot handle NaNs.
    combined_X_y = pd.concat([X, y], axis=1).dropna()
    X_cleaned = combined_X_y[['lat', 'lon', 'timestamp_hour']]
    y_cleaned = combined_X_y[['wind_speed', 'wind_direction']]

    if not X_cleaned.empty and not y_cleaned.empty:
        # Split the data into training and testing sets
        # Using 80% for training and 20% for testing, with a random state for reproducibility
        X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

        # Print the shapes of the resulting sets
        print("Shape of X_train:", X_train.shape)
        print("Shape of X_test:", X_test.shape)
        print("Shape of y_train:", y_train.shape)
        print("Shape of y_test:", y_test.shape)

    else:
        print("No valid data points available for splitting after cleaning.")
        # Define empty dataframes for the split results to avoid errors in subsequent cells
        X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()


else:
    print("df_combined DataFrame not found or is empty. Cannot perform data splitting. Please run previous data preparation steps.")
    # Define empty dataframes for the split results to avoid errors in subsequent cells
    X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

Shape of X_train: (2400, 3)
Shape of X_test: (600, 3)
Shape of y_train: (2400, 2)
Shape of y_test: (600, 2)


## Train ml model

### Subtask:
Train the chosen Machine Learning model (Random Forest Regressor) on the training data.


**Reasoning**:
Import the Random Forest Regressor model and train it on the training data.



In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate the Random Forest Regressor model
# Using random_state for reproducibility
model = RandomForestRegressor(random_state=42)

# Train the model using the training data
model.fit(X_train, y_train)

print("Random Forest Regressor model trained successfully.")

Random Forest Regressor model trained successfully.


**Reasoning**:
The previous step failed because the training data was not defined. This is because the data splitting step failed in the previous turn. The data splitting step failed because the dataframe `df_combined` was empty. I will attempt to reload the data and perform the feature engineering and splitting steps again to ensure the data is available before training the model.



In [ ]:
import pandas as pd
import json
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# --- Step 1: Load ALL Balloon Data ---
# Ensure this file exists by running the balloon data acquisition cell (I38C1hkKEBQZ) first
try:
    with open('windborne_balloon_data.json', 'r', encoding='utf-8') as f:
        all_balloon_data = json.load(f)
except FileNotFoundError:
    print("Error: windborne_balloon_data.json not found. Please run the balloon data acquisition step (cell I38C1hkKEBQZ) first.")
    all_balloon_data = []
except json.JSONDecodeError:
    print("Error: Could not decode windborne_balloon_data.json. The file might be corrupted.")
    all_balloon_data = []
except Exception as e:
    print(f"An unexpected error occurred while loading windborne_balloon_data.json: {e}")
    all_balloon_data = []

print(f"Loaded {len(all_balloon_data)} total balloon position entries.")

# --- Step 2: Data Combination (Fetch Weather Data for ALL Data and Combine) ---
# This assumes the get_weather_data function (cell 00963023) is defined and works.
# Ensure get_weather_data function is defined
if 'get_weather_data' not in locals():
     print("Error: get_weather_data function is not defined. Please ensure the external data acquisition step (cell 00963023) was run successfully.")
     combined_data = all_balloon_data # Keep original data if weather fetching is not possible
else:
    combined_data = []
    weather_fetch_count = 0
    successful_fetches = 0
    API_CALL_DELAY = 0.1 # seconds delay between weather API calls

    if all_balloon_data: # Proceed only if the data is not empty
        print("Starting data combination with weather data for the entire dataset...")
        for i, entry in enumerate(all_balloon_data):
            lat = entry.get('lat')
            lon = entry.get('lon')

            # Ensure lat and lon are floats before passing to get_weather_data
            try:
                lat = float(lat) if lat is not None else None
                lon = float(lon) if lon is not None else None
            except (ValueError, TypeError):
                print(f"Skipping weather fetch for entry {i} due to invalid lat/lon type: lat={lat}, lon={lon}")
                lat = None
                lon = None


            if lat is not None and lon is not None:
                # Fetch weather data for the balloon's location
                # This will use userdata.get() internally based on our previous modification
                wind_speed, wind_direction = get_weather_data(lat, lon)
                weather_fetch_count += 1

                # Add weather data to the balloon entry
                if wind_speed is not None and wind_direction is not None:
                    entry['wind_speed'] = wind_speed
                    entry['wind_direction'] = wind_direction
                    successful_fetches += 1
                else:
                    # Handle cases where weather data could not be fetched for this specific point
                    # print(f"Could not fetch weather data for entry {i} at lat={lat}, lon={lon}") # Optional: keep print for debugging
                    entry['wind_speed'] = None
                    entry['wind_direction'] = None

                combined_data.append(entry)

                # Add a delay after each API call to respect rate limits
                time.sleep(API_CALL_DELAY)

            else:
                # If lat/lon is missing or invalid, we cannot fetch weather, so we add the entry without weather data.
                combined_data.append(entry)

        print(f"Attempted to fetch weather data for {weather_fetch_count} entries in the dataset.")
        print(f"Successfully fetched weather data for {successful_fetches} entries in the dataset.")
        print(f"Combined data contains {len(combined_data)} entries.")
    else:
        print("Skipping data combination as the balloon data is empty.")

# --- Step 3: Prepare Data for Feature Engineering ---
if combined_data:
    # Convert the list of dictionaries to a pandas DataFrame
    df_combined = pd.DataFrame(combined_data)

    # Ensure latitude and longitude columns are numeric
    df_combined['lat'] = pd.to_numeric(df_combined['lat'], errors='coerce')
    df_combined['lon'] = pd.to_numeric(df_combined['lon'], errors='coerce')
    df_combined['wind_speed'] = pd.to_numeric(df_combined['wind_speed'], errors='coerce')
    df_combined['wind_direction'] = pd.to_numeric(df_combined['wind_direction'], errors='coerce')

    # Drop rows where lat or lon are not valid numbers after coercion
    df_combined.dropna(subset=['lat', 'lon', 'wind_speed', 'wind_direction'], inplace=True)

    print(f"DataFrame created with {len(df_combined)} entries after cleaning.")

    # --- Step 4: Feature Engineering for ML ---

    # Ensure data is sorted by balloon ID and timestamp for correct difference calculation
    # Assuming 'timestamp' is a string like '00H_ago', '01H_ago', etc. Convert to a sortable format.
    # We can extract the hour number for sorting.
    df_combined['timestamp_hour'] = df_combined['timestamp'].str.extract(r'(\d+)H_ago').astype(float)
    df_combined = df_combined.sort_values(by=['id', 'timestamp_hour'])

    # Calculate the difference in latitude and longitude for consecutive points of the same balloon
    df_combined['delta_lat'] = df_combined.groupby('id')['lat'].diff()
    df_combined['delta_lon'] = df_combined.groupby('id')['lon'].diff()

    # Calculate the time difference in hours (assuming timestamps like '00H_ago', '01H_ago' are 1 hour apart)
    # If the timestamp format is consistent 'XXH_ago', the difference in 'timestamp_hour' is the time difference in hours.
    df_combined['time_diff_hours'] = df_combined.groupby('id')['timestamp_hour'].diff()

    # Calculate the displacement (Euclidean distance)
    df_combined['displacement'] = (df_combined['delta_lat']**2 + df_combined['delta_lon']**2)**0.5

    # Calculate approximate speed (displacement / time_diff)
    # Avoid division by zero or NaN time_diff
    df_combined['speed_approx'] = df_combined['displacement'] / df_combined['time_diff_hours']

    # Handle missing values created by diff() (the first entry for each balloon)
    # These rows cannot be used to predict wind based on previous movement.
    # Based on the previous turn's error, it seems dropping these rows resulted in an empty df.
    # Re-evaluate dropping. If we are predicting wind based on location/time, we don't need the diff features.
    # Let's keep rows that have valid lat, lon, timestamp_hour, wind_speed, and wind_direction.
    # We will use lat, lon, timestamp_hour as features to predict wind_speed and wind_direction.

    # Define features (X) and target variables (y) based on the redefined problem
    X = df_combined[['lat', 'lon', 'timestamp_hour']]
    y = df_combined[['wind_speed', 'wind_direction']]

    # Drop rows with any NaN values in X or y before splitting
    # This ensures we only train on complete data points for the chosen features and targets.
    combined_X_y = pd.concat([X, y], axis=1).dropna()
    X_cleaned = combined_X_y[['lat', 'lon', 'timestamp_hour']]
    y_cleaned = combined_X_y[['wind_speed', 'wind_direction']]

    print(f"Data points available for ML after dropping NaNs: {len(X_cleaned)}")


    # --- Step 5: Split the data into training and testing sets ---
    if not X_cleaned.empty and not y_cleaned.empty:
        X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

        # Print the shapes of the resulting sets
        print("Shape of X_train:", X_train.shape)
        print("Shape of X_test:", X_test.shape)
        print("Shape of y_train:", y_train.shape)
        print("Shape of y_test:", y_test.shape)

        # --- Step 6: Train the Machine Learning model ---
        # Instantiate the Random Forest Regressor model
        model = RandomForestRegressor(random_state=42)

        # Train the model using the training data
        model.fit(X_train, y_train)

        print("Random Forest Regressor model trained successfully.")

    else:
        print("No valid data points available for training after cleaning. Skipping model training.")


else:
    print("Cannot perform feature engineering or model training as no balloon data was loaded or combined.")


Loaded 3000 total balloon position entries.
Starting data combination with weather data for the entire dataset...
Attempted to fetch weather data for 3000 entries in the dataset.
Successfully fetched weather data for 3000 entries in the dataset.
Combined data contains 3000 entries.
DataFrame created with 3000 entries after cleaning.
Data points available for ML after dropping NaNs: 3000
Shape of X_train: (2400, 3)
Shape of X_test: (600, 3)
Shape of y_train: (2400, 2)
Shape of y_test: (600, 2)
Random Forest Regressor model trained successfully.


## Evaluate model

### Subtask:
Evaluate the performance of the trained Random Forest Regressor model using the testing data.


**Reasoning**:
Evaluate the performance of the trained Random Forest Regressor model using the testing data by calculating MSE and R-squared for both wind speed and wind direction predictions.



In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on the test data
y_pred = model.predict(X_test)

# The predictions y_pred will be a numpy array with two columns: wind_speed and wind_direction
y_pred_speed = y_pred[:, 0]
y_pred_direction = y_pred[:, 1]

# Extract the actual values from y_test (which is a pandas DataFrame)
y_test_speed = y_test['wind_speed']
y_test_direction = y_test['wind_direction']

# Calculate Mean Squared Error (MSE)
mse_speed = mean_squared_error(y_test_speed, y_pred_speed)
mse_direction = mean_squared_error(y_test_direction, y_pred_direction)

# Calculate R-squared score
r2_speed = r2_score(y_test_speed, y_pred_speed)
r2_direction = r2_score(y_test_direction, y_pred_direction)

# Print the evaluation metrics
print("Model Evaluation Results on Test Data:")
print(f"Wind Speed - MSE: {mse_speed:.4f}")
print(f"Wind Speed - R-squared: {r2_speed:.4f}")
print(f"Wind Direction - MSE: {mse_direction:.4f}")
print(f"Wind Direction - R-squared: {r2_direction:.4f}")

Model Evaluation Results on Test Data:
Wind Speed - MSE: 4.2253
Wind Speed - R-squared: 0.6880
Wind Direction - MSE: 5306.2231
Wind Direction - R-squared: 0.4093


## Make predictions/gain insights

### Subtask:
Use the trained Random Forest Regressor model to make predictions and gain insights into the relationships between location/time and wind conditions.


**Reasoning**:
Use the trained model to make predictions on the test data, separate the predictions, analyze feature importances, create a DataFrame for importance, and print the feature importances.



In [ ]:
# 1. Use the trained model to make predictions on the test features (X_test)
# The predictions were already made in the previous step and stored in y_pred
# y_pred = model.predict(X_test) # This line is commented out as it was done in the previous step

# 2. Separate the predictions into two variables (y_pred_speed and y_pred_direction)
# This separation was also done in the previous step
# y_pred_speed = y_pred[:, 0] # This line is commented out
# y_pred_direction = y_pred[:, 1] # This line is commented out

# 3. Analyze the feature importance of the trained model
# Access the feature importances using model.feature_importances_
feature_importances = model.feature_importances_

# 4. Create a pandas DataFrame from the feature_importances_ array, mapping the importance scores back to the feature names from X_train. Sort this DataFrame.
# Get the feature names from X_train
feature_names = X_train.columns

# Create the DataFrame
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# 5. Print the feature importances
print("\nFeature Importances from Random Forest Regressor:")
display(feature_importance_df)


Feature Importances from Random Forest Regressor:


,feature,importance
0,lat,0.499677
1,lon,0.462815
2,timestamp_hour,0.037508


## Visualize ml results

### Subtask:
Visualize the predictions or insights from the ML model, potentially overlaying them on the balloon trajectories or maps.


**Reasoning**:
Combine test features, actual, and predicted wind values into a single DataFrame and prepare for visualization.



In [ ]:
import pandas as pd

# 1. Create a new DataFrame combining the test features (X_test), the actual wind speed and direction (y_test), and the predicted wind speed and direction (y_pred).
# Ensure the indices align correctly. X_test and y_test already have aligned indices.
# y_pred is a numpy array, so we need to convert it to a DataFrame with matching indices.

# Create DataFrames for predictions with the same index as X_test/y_test
y_pred_df = pd.DataFrame(y_pred, index=X_test.index, columns=['predicted_wind_speed', 'predicted_wind_direction'])

# Concatenate X_test, y_test, and the predicted values DataFrames
# Use axis=1 to concatenate columns
visualization_df = pd.concat([X_test, y_test, y_pred_df], axis=1)

# 2. Rename the columns in the combined DataFrame for clarity (already done in y_pred_df columns)
# Ensure column names are clear
visualization_df.rename(columns={
    'wind_speed': 'actual_wind_speed',
    'wind_direction': 'actual_wind_direction'
}, inplace=True)

# 3. Create a scatter plot on a map using plotly.express to visualize the test locations.
# 4. Color the points based on the difference between predicted and actual wind speed or wind direction to highlight areas where the model performs well or poorly.
# Calculate the difference in wind speed and wind direction
visualization_df['speed_difference'] = visualization_df['predicted_wind_speed'] - visualization_df['actual_wind_speed']
visualization_df['direction_difference'] = visualization_df['predicted_wind_direction'] - visualization_df['actual_wind_direction']

# Visualize using speed difference for coloring
fig_speed_diff = px.scatter_mapbox(visualization_df,
                                   lat="lat",
                                   lon="lon",
                                   color="speed_difference", # Color by wind speed difference
                                   size="actual_wind_speed", # Size by actual wind speed (optional, or could size by magnitude of difference)
                                   hover_name=visualization_df.index, # Use index for hover name, which corresponds to the original row
                                   hover_data={
                                       "lat": True,
                                       "lon": True,
                                       "timestamp_hour": True,
                                       "actual_wind_speed": ':.2f',
                                       "predicted_wind_speed": ':.2f',
                                       "speed_difference": ':.2f',
                                       "actual_wind_direction": True,
                                       "predicted_wind_direction": True,
                                       "direction_difference": ':.2f'
                                       },
                                   color_continuous_scale="RdBu", # Red-Blue colormap for divergence (difference)
                                   size_max=10, # Adjust size based on data distribution
                                   zoom=1,
                                   height=600)

# 6. Configure the map style and layout for better visualization.
fig_speed_diff.update_layout(mapbox_style="open-street-map")
fig_speed_diff.update_layout(margin={"r":0,"t":40,"l":0,"b":0}) # Add some top margin for the title
fig_speed_diff.update_layout(title_text="Wind Speed Prediction Difference (Predicted - Actual) on Test Data") # Add a title

# 7. Display the plot.
fig_speed_diff.show()

# Optional: Visualize using direction difference for coloring
# Note: Wind direction is circular, simple difference might not be ideal for visualization.
# A more advanced approach would handle the 0/360 degree wrap-around.
# For this visualization, we'll use the simple difference but acknowledge its limitation for direction.
fig_direction_diff = px.scatter_mapbox(visualization_df,
                                   lat="lat",
                                   lon="lon",
                                   color="direction_difference", # Color by wind direction difference
                                   size="actual_wind_direction", # Size by actual wind direction (less intuitive than speed)
                                   hover_name=visualization_df.index, # Use index for hover name
                                   hover_data={
                                       "lat": True,
                                       "lon": True,
                                       "timestamp_hour": True,
                                       "actual_wind_speed": ':.2f',
                                       "predicted_wind_speed": ':.2f',
                                       "speed_difference": ':.2f',
                                       "actual_wind_direction": True,
                                       "predicted_wind_direction": True,
                                       "direction_difference": ':.2f'
                                       },
                                   color_continuous_scale="RdBu",
                                   size_max=10,
                                   zoom=1,
                                   height=600)

fig_direction_diff.update_layout(mapbox_style="open-street-map")
fig_direction_diff.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_direction_diff.update_layout(title_text="Wind Direction Prediction Difference (Predicted - Actual) on Test Data")
fig_direction_diff.show()

# 8. Optionally, create additional visualizations (e.g., scatter plots comparing actual vs. predicted values)
fig_speed_scatter = px.scatter(visualization_df,
                               x="actual_wind_speed",
                               y="predicted_wind_speed",
                               hover_data={"lat": True, "lon": True, "timestamp_hour": True, "speed_difference": True},
                               title="Actual vs. Predicted Wind Speed")
fig_speed_scatter.show()

fig_direction_scatter = px.scatter(visualization_df,
                                  x="actual_wind_direction",
                                  y="predicted_wind_direction",
                                  hover_data={"lat": True, "lon": True, "timestamp_hour": True, "direction_difference": True},
                                  title="Actual vs. Predicted Wind Direction")
fig_direction_scatter.show()

# Task
Combine the extracted Windborne balloon data and OpenWeatherMap data, and then apply a Machine Learning technique to gain insights or make predictions, such as predicting wind fields. Visualize the results and ensure the process can dynamically update with new data.

## Potential Insights and Problems to Tackle

Based on the combined balloon position data and OpenWeatherMap wind data, we can explore several interesting insights and problems:

*   **Understanding Wind Influence:** Analyze how wind speed and direction correlate with balloon movement and trajectories at different locations and times.
*   **Predicting Wind Fields:** Build a model to predict wind speed and direction at unobserved locations or future times based on historical balloon positions and weather data. This could be a core ML problem.
*   **Identifying Anomalies:** Detect unusual balloon behavior that might not be explained by local wind conditions, potentially indicating other atmospheric phenomena or balloon-specific issues.
*   **Optimizing Balloon Deployment/Navigation:** (More advanced) If we could predict wind, we might be able to suggest optimal launch times or altitudes for future balloon missions to reach certain areas.

## Dynamic Updating

Since both the Windborne API and weather APIs provide relatively current data, the process we build should be designed to update dynamically. This means:

*   The data acquisition steps for both balloon and weather data should fetch the latest available information (e.g., the last 24 hours for Windborne).
*   The data combination, ML model training (or prediction), and visualization steps should be executable with this newly fetched data to provide up-to-date insights or predictions.

## External API Choice Justification

We have chosen to combine the balloon position data with weather data, specifically wind speed and direction, obtained from the OpenWeatherMap API. This choice was made due to OpenWeatherMap's **relevance** (wind directly affects balloons), **accessibility** (publicly available API), and the **potential for interesting analysis** when correlating balloon movements with local wind conditions.

## Define Web Application Requirements

### Subtask:
Determine what the interactive webpage should display and allow users to do (e.g., visualize the combined data, show ML predictions, allow user input for predictions).

**Reasoning**:
Prompt the user to define the specific functionalities and content they want for the public web application. This will guide the selection of frameworks and development steps.

Please describe what you envision for the interactive webpage:

*   What data or visualizations should be displayed? (e.g., the map with balloon positions and wind data, ML model evaluation results, feature importances)
*   Should users be able to interact with the data or model? (e.g., filter data by time/location, input coordinates to get wind predictions)
*   What is the primary goal of the webpage? (e.g., showcase the data, demonstrate the ML model, provide a tool for users)

Your input here will help shape the development of the web application.

In [ ]:
# Install Flask
%pip install Flask pandas plotly scikit-learn

In [ ]:
from flask import Flask, render_template_string, request
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
import os
import time
from google.colab import userdata # Assuming we'll continue using userdata for API key
import pickle # Needed to save/load the trained ML model

# Define the Flask application
app = Flask(__name__)

# --- Global variables and data loading (will be loaded once when the app starts) ---
# In a real application, you might load data from a database or file storage.
# For this example, we'll assume the combined data is available in a JSON file.
# You would need to run the data acquisition and combination steps beforehand
# to create 'windborne_balloon_weather_combined.json' or similar.

combined_data = []
df_combined = pd.DataFrame()
ml_model = None # Placeholder for the trained ML model

# --- Data Loading and Model Loading Functions ---

def load_combined_data():
    """Loads combined balloon and weather data from a JSON file."""
    global combined_data, df_combined
    try:
        # Assuming the combined data with weather is saved to this file after the combination step
        with open('windborne_balloon_weather_combined.json', 'r', encoding='utf-8') as f:
            combined_data = json.load(f)
        df_combined = pd.DataFrame(combined_data)
        # Ensure numeric types and handle NaNs as needed for ML
        df_combined['lat'] = pd.to_numeric(df_combined['lat'], errors='coerce')
        df_combined['lon'] = pd.to_numeric(df_combined['lon'], errors='coerce')
        df_combined['wind_speed'] = pd.to_numeric(df_combined['wind_speed'], errors='coerce')
        df_combined['wind_direction'] = pd.to_numeric(df_combined['wind_direction'], errors='coerce')
        # Add timestamp_hour and movement features if needed for the ML model
        if 'timestamp' in df_combined.columns:
             df_combined['timestamp_hour'] = df_combined['timestamp'].str.extract(r'(\d+)H_ago').astype(float)
        # Drop rows with essential missing data for ML
        df_combined.dropna(subset=['lat', 'lon', 'timestamp_hour', 'wind_speed', 'wind_direction'], inplace=True) # Ensure essential columns for ML are not NaN

        print(f"Loaded {len(df_combined)} combined data entries.")

    except FileNotFoundError:
        print("Error loading combined data: 'windborne_balloon_weather_combined.json' not found.")
    except json.JSONDecodeError:
        print("Error decoding combined data JSON.")
    except Exception as e:
        print(f"An error occurred loading combined data: {e}")

def load_ml_model():
    """Loads the trained ML model from a file."""
    global ml_model
    try:
        # Assuming the trained model is saved to 'ml_model.pkl'
        with open('ml_model.pkl', 'rb') as f:
            ml_model = pickle.load(f)
        print("ML model loaded successfully.")
    except FileNotFoundError:
        print("Error loading ML model: 'ml_model.pkl' not found. Please train and save the model first.")
    except Exception as e:
        print(f"An error occurred loading ML model: {e}")


# --- Flask Routes ---

@app.route('/')
def index():
    """Home page with a simple message."""
    return "Welcome to the Balloon and Weather Data App!"

@app.route('/data')
def show_data():
    """Displays a simple table of the loaded combined data."""
    if not df_combined.empty:
        # Render a simple HTML table of the first few rows
        return render_template_string(df_combined.head().to_html())
    else:
        return "No combined data loaded."

@app.route('/predict', methods=['GET', 'POST'])
def predict_wind():
    """Provides a simple interface to get wind predictions."""
    if request.method == 'POST':
        # Get input from form (example: expecting lat, lon, hour)
        try:
            lat = float(request.form['lat'])
            lon = float(request.form['lon'])
            hour = float(request.form['hour']) # Assuming hour is a number 0-23

            if ml_model:
                # Prepare input for the model (needs to match training features)
                # Our model used ['lat', 'lon', 'timestamp_hour']
                input_data = pd.DataFrame([[lat, lon, hour]], columns=['lat', 'lon', 'timestamp_hour'])
                prediction = ml_model.predict(input_data)
                predicted_speed = prediction[0, 0]
                predicted_direction = prediction[0, 1]

                return f"Predicted Wind Speed: {predicted_speed:.2f} m/s, Predicted Wind Direction: {predicted_direction:.2f} degrees"
            else:
                return "ML model not loaded. Cannot make predictions."
        except ValueError:
            return "Invalid input. Please provide numeric values for latitude, longitude, and hour."
        except Exception as e:
            return f"An error occurred during prediction: {e}"

    else:
        # Display a simple form for GET requests
        form_html = """
        <h2>Get Wind Prediction</h2>
        <form method="post">
            Latitude: <input type="text" name="lat"><br>
            Longitude: <input type="text" name="lon"><br>
            Hour (0-23): <input type="text" name="hour"><br>
            <input type="submit" value="Predict">
        </form>
        """
        return form_html

@app.route('/visualization')
def show_visualization():
    """Generates and displays the Plotly visualization."""
    if not df_combined.empty:
        fig = px.scatter_mapbox(df_combined,
                                lat="lat",
                                lon="lon",
                                color="wind_speed",
                                size="wind_speed",
                                hover_name="id",
                                hover_data={"lat": True, "lon": True, "wind_speed": True, "wind_direction": True, "timestamp": True},
                                color_continuous_scale=px.colors.cyclical.IceFire,
                                size_max=15,
                                zoom=1,
                                height=500)

        fig.update_layout(mapbox_style="open-street-map")
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.update_layout(title_text=f"Balloon Positions ({len(df_combined)} entries) with Wind Speed")

        # Convert the Plotly figure to HTML
        graph_html = fig.to_html(full_html=False)

        # Embed the graph HTML in a simple page
        page_html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Balloon Data Visualization</title>
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        </head>
        <body>
            <h1>Balloon Data Visualization</h1>
            {graph_html}
        </body>
        </html>
        """
        return page_html

    else:
        return "No combined data available for visualization."


# --- Run the app (for development in Colab) ---
# This part is specific to running in Colab. For actual deployment,
# you would use a production web server (like Gunicorn or uWSGI).

# Load data and model when the application starts
load_combined_data()
load_ml_model()

# To run in Colab, you'll typically use ngrok or colab-aistudio
# from flask_ngrok import run_with_ngrok
# run_with_ngrok(app)

# Or using colab-aistudio:
# from colab_aistudio import run_with_aistudio
# run_with_aistudio(app)

# app.run() # Use this for local development or with a production server

# Note: Running Flask directly in Colab with app.run() without a tunnel
# will not make it publicly accessible. You need ngrok or colab-aistudio
# for public access from Colab.

Error loading combined data: 'windborne_balloon_weather_combined.json' not found.
Error loading ML model: 'ml_model.pkl' not found. Please train and save the model first.


## Prepare data for ml

### Subtask:
Format the combined balloon and weather data into a structure suitable for Machine Learning. This might involve creating features from the existing data, such as changes in balloon position, time differences, and corresponding wind conditions.

**Reasoning**:
The combined data is currently in a list of dictionaries (`combined_data` from the previous step) or can be loaded from `windborne_balloon_weather_combined.json`. To prepare it for Machine Learning, I need to convert it into a pandas DataFrame and then create relevant features. Based on the project goal of potentially predicting wind fields or understanding wind influence, features related to balloon movement (change in lat/lon, time difference, speed) could be valuable inputs, along with location and time and potentially the observed wind itself as input features to predict wind at other locations/times. I will calculate the change in latitude and longitude, the time difference, and the balloon's displacement and speed. I will also ensure necessary columns like wind speed and direction are numeric.

In [ ]:
import pandas as pd
import json

# --- Load combined data ---
# This cell assumes that the initial balloon data acquisition (cell 1e09f5b1)
# has been run to create 'windborne_balloon_data.json'.
# It also assumes that the get_weather_data function (cell 00963023) is defined.

# Load the *full* balloon data first
try:
    with open('windborne_balloon_data.json', 'r', encoding='utf-8') as f:
        all_balloon_data = json.load(f)
except FileNotFoundError:
    print("Error: windborne_balloon_data.json not found. Please ensure the initial data acquisition was successful.")
    all_balloon_data = []
except json.JSONDecodeError:
    print("Error: Could not decode windborne_balloon_data.json. The file might be corrupted.")
    all_balloon_data = []
except Exception as e:
    print(f"An unexpected error occurred while loading windborne_balloon_data.json: {e}")
    all_balloon_data = []

print(f"Loaded {len(all_balloon_data)} total balloon position entries for full combination.")

# --- Data Combination (Fetch Weather Data for ALL Data and Combine) ---
# Ensure get_weather_data function is defined
if 'get_weather_data' not in locals():
     print("Error: get_weather_data function is not defined. Please ensure the external data acquisition step (cell 00963023) was run successfully.")
     combined_data_full = all_balloon_data # Keep original data if weather fetching is not possible
else:
    combined_data_full = []
    weather_fetch_count = 0
    successful_fetches = 0
    API_CALL_DELAY = 0.1 # seconds delay between weather API calls

    if all_balloon_data: # Proceed only if the data is not empty
        print("Starting data combination with weather data for the entire dataset...")
        for i, entry in enumerate(all_balloon_data):
            lat = entry.get('lat')
            lon = entry.get('lon')

            # Ensure lat and lon are floats before passing to get_weather_data
            try:
                lat = float(lat) if lat is not None else None
                lon = float(lon) if lon is not None else None
            except (ValueError, TypeError):
                print(f"Skipping weather fetch for entry {i} due to invalid lat/lon type: lat={lat}, lon={lon}")
                lat = None
                lon = None


            if lat is not None and lon is not None:
                # Fetch weather data for the balloon's location
                # This will use userdata.get() internally based on our previous modification
                wind_speed, wind_direction = get_weather_data(lat, lon)
                weather_fetch_count += 1

                # Add weather data to the balloon entry
                if wind_speed is not None and wind_direction is not None:
                    entry['wind_speed'] = wind_speed
                    entry['wind_direction'] = wind_direction
                    successful_fetches += 1
                else:
                    # Handle cases where weather data could not be fetched for this specific point
                    # print(f"Could not fetch weather data for entry {i} at lat={lat}, lon={lon}") # Optional: keep print for debugging
                    entry['wind_speed'] = None
                    entry['wind_direction'] = None

                combined_data_full.append(entry)

                # Add a delay after each API call to respect rate limits
                time.sleep(API_CALL_DELAY)

            else:
                # If lat/lon is missing or invalid, we cannot fetch weather, so we add the entry without weather data.
                combined_data_full.append(entry)

        print(f"Attempted to fetch weather data for {weather_fetch_count} entries in the dataset.")
        print(f"Successfully fetched weather data for {successful_fetches} entries in the dataset.")
        print(f"Combined data contains {len(combined_data_full)} entries.")
    else:
        print("Skipping data combination as the balloon data is empty.")


# --- Step 3: Prepare Data for Feature Engineering ---
if combined_data_full:
    # Convert the list of dictionaries to a pandas DataFrame
    df_combined = pd.DataFrame(combined_data_full)

    # Ensure latitude and longitude columns are numeric
    df_combined['lat'] = pd.to_numeric(df_combined['lat'], errors='coerce')
    df_combined['lon'] = pd.to_numeric(df_combined['lon'], errors='coerce')
    df_combined['wind_speed'] = pd.to_numeric(df_combined['wind_speed'], errors='coerce')
    df_combined['wind_direction'] = pd.to_numeric(df_combined['wind_direction'], errors='coerce')

    # Drop rows where lat or lon are not valid numbers after coercion, and also drop if wind data is missing
    df_combined.dropna(subset=['lat', 'lon', 'wind_speed', 'wind_direction'], inplace=True)

    print(f"DataFrame created with {len(df_combined)} entries after cleaning for ML features.")

    # --- Step 4: Feature Engineering for ML (Movement-based features - Optional depending on ML problem) ---
    # This part is relevant if we were predicting wind based on balloon movement.
    # Based on the redefined ML problem (predicting wind at a location/time),
    # these movement features are NOT the primary inputs.
    # However, let's keep the calculation here in case they are needed for other analysis or future ML problems.
    # We will use lat, lon, timestamp_hour as features for the current ML problem.

    # Ensure data is sorted by balloon ID and timestamp for correct difference calculation
    # Assuming 'timestamp' is a string like '00H_ago', '01H_ago', etc. Convert to a sortable format.
    # We can extract the hour number for sorting.
    if 'timestamp' in df_combined.columns:
        df_combined['timestamp_hour'] = df_combined['timestamp'].str.extract(r'(\d+)H_ago').astype(float)
        df_combined = df_combined.sort_values(by=['id', 'timestamp_hour'])

        # Calculate the difference in latitude and longitude for consecutive points of the same balloon
        df_combined['delta_lat'] = df_combined.groupby('id')['lat'].diff()
        df_combined['delta_lon'] = df_combined.groupby('id')['lon'].diff()

        # Calculate the time difference in hours (assuming timestamps like '00H_ago', '01H_ago' are 1 hour apart)
        # If the timestamp format is consistent 'XXH_ago', the difference in 'timestamp_hour' is the time difference in hours.
        df_combined['time_diff_hours'] = df_combined.groupby('id')['timestamp_hour'].diff()

        # Calculate the displacement (Euclidean distance)
        df_combined['displacement'] = (df_combined['delta_lat']**2 + df_combined['delta_lon']**2)**0.5

        # Calculate approximate speed (displacement / time_diff)
        # Avoid division by zero or NaN time_diff
        df_combined['speed_approx'] = df_combined['displacement'] / df_combined['time_diff_hours']

        # Note: We are NOT dropping rows with NaN movement features here if we are predicting
        # wind based on location/time, as those rows are still valuable data points
        # even if we can't calculate their movement. We will drop NaNs for the specific
        # features used in the ML model in the splitting step.
    else:
        print("Warning: 'timestamp' column not found for feature engineering.")
        df_combined['timestamp_hour'] = None


    print(f"DataFrame after potential feature engineering: {len(df_combined)} entries.")

    # Display the first few rows with the new features
    display(df_combined.head())

else:
    print("Cannot prepare data for ML as no combined data was loaded.")
    df_combined = pd.DataFrame() # Ensure df_combined is defined even if empty

Loaded 3000 total balloon position entries for full combination.
Starting data combination with weather data for the entire dataset...
Attempted to fetch weather data for 3000 entries in the dataset.
Successfully fetched weather data for 3000 entries in the dataset.
Combined data contains 3000 entries.
DataFrame created with 3000 entries after cleaning for ML features.
DataFrame after potential feature engineering: 3000 entries.


,id,lat,lon,timestamp,raw,wind_speed,wind_direction,timestamp_hour,delta_lat,delta_lon,time_diff_hours,displacement,speed_approx
0,balloon_00_0,70.789291,37.272190,00H_ago,"[70.78929088655421, 37.27219027444302, 2.70364...",5.69,263,0.0,NaN,NaN,NaN,NaN,NaN
1,balloon_00_1,47.252662,-62.017661,00H_ago,"[47.25266228831717, -62.017660533384024, 16.38...",7.06,218,0.0,NaN,NaN,NaN,NaN,NaN
10,balloon_00_10,40.764657,-170.857804,00H_ago,"[40.76465653049449, -170.85780403006524, 18.04...",8.08,204,0.0,NaN,NaN,NaN,NaN,NaN
100,balloon_00_100,0.000000,0.000000,00H_ago,"[0, 0, 0]",7.14,171,0.0,NaN,NaN,NaN,NaN,NaN
101,balloon_00_101,49.774230,31.812494,00H_ago,"[49.77423033626831, 31.81249406339161, 21.9972...",3.10,303,0.0,NaN,NaN,NaN,NaN,NaN


## Train ML Model (and Save)

### Subtask:
Train the chosen Machine Learning model (Random Forest Regressor) on the training data and save the trained model to a file (`ml_model.pkl`) so it can be loaded by the Flask application.

**Reasoning**:
Train the Random Forest Regressor model on the training data (`X_train`, `y_train`) and then use the `pickle` library to save the trained model object to a file named `ml_model.pkl`. This file will be loaded by the Flask application when it starts. This assumes that the previous steps to prepare and split the data (`X_train`, `y_train`) have been successfully executed.

In [ ]:
import pickle
from sklearn.ensemble import RandomForestRegressor

# --- Ensure training data is available ---
if 'X_train' in locals() and not X_train.empty and 'y_train' in locals() and not y_train.empty:
    # Instantiate the Random Forest Regressor model
    # Using random_state for reproducibility
    model = RandomForestRegressor(random_state=42)

    # Train the model using the training data
    model.fit(X_train, y_train)

    print("Random Forest Regressor model trained successfully.")

    # --- Save the trained model to a file ---
    try:
        with open('ml_model.pkl', 'wb') as f:
            pickle.dump(model, f)
        print("Trained ML model saved to ml_model.pkl")
    except Exception as e:
        print(f"Error saving ML model: {e}")
else:
    print("Skipping model training and saving as training data (X_train or y_train) is not available or is empty.")

Random Forest Regressor model trained successfully.
Trained ML model saved to ml_model.pkl


## Evaluate model

### Subtask:
Evaluate the performance of the trained Random Forest Regressor model using the testing data.

**Reasoning**:
Evaluate the performance of the trained Random Forest Regressor model using the testing data by calculating MSE and R-squared for both wind speed and wind direction predictions.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Ensure the model and test data are available
if 'model' in locals() and 'X_test' in locals() and not X_test.empty and 'y_test' in locals() and not y_test.empty:
    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # The predictions y_pred will be a numpy array with two columns: wind_speed and wind_direction
    y_pred_speed = y_pred[:, 0]
    y_pred_direction = y_pred[:, 1]

    # Extract the actual values from y_test (which is a pandas DataFrame)
    y_test_speed = y_test['wind_speed']
    y_test_direction = y_test['wind_direction']

    # Calculate Mean Squared Error (MSE)
    mse_speed = mean_squared_error(y_test_speed, y_pred_speed)
    mse_direction = mean_squared_error(y_test_direction, y_pred_direction)

    # Calculate R-squared score
    r2_speed = r2_score(y_test_speed, y_pred_speed)
    r2_direction = r2_score(y_test_direction, y_pred_direction)

    # Print the evaluation metrics
    print("Model Evaluation Results on Test Data:")
    print(f"Wind Speed - MSE: {mse_speed:.4f}")
    print(f"Wind Speed - R-squared: {r2_speed:.4f}")
    print(f"Wind Direction - MSE: {mse_direction:.4f}")
    print(f"Wind Direction - R-squared: {r2_direction:.4f}")

else:
    print("Model or test data not available for evaluation. Please ensure the model was trained and data was split successfully.")

Model Evaluation Results on Test Data:
Wind Speed - MSE: 4.0247
Wind Speed - R-squared: 0.6992
Wind Direction - MSE: 4894.2232
Wind Direction - R-squared: 0.4637


## Make predictions/gain insights

### Subtask:
Use the trained Random Forest Regressor model to make predictions and gain insights into the relationships between location/time and wind conditions.

**Reasoning**:
Use the trained model to make predictions on the test data, separate the predictions, analyze feature importances, create a DataFrame for importance, and print the feature importances.

In [ ]:
# Ensure the trained model and test data are available
if 'model' in locals() and 'X_test' in locals() and not X_test.empty:

    # 1. Use the trained model to make predictions on the test features (X_test)
    y_pred = model.predict(X_test)

    # 2. Separate the predictions into two variables (y_pred_speed and y_pred_direction)
    y_pred_speed = y_pred[:, 0]
    y_pred_direction = y_pred[:, 1]

    # 3. Analyze the feature importance of the trained model
    # Access the feature importances using model.feature_importances_
    feature_importances = model.feature_importances_

    # 4. Create a pandas DataFrame from the feature_importances_ array, mapping the importance scores back to the feature names from X_train. Sort this DataFrame.
    # Ensure X_train is available to get feature names
    if 'X_train' in locals() and not X_train.empty:
        feature_names = X_train.columns

        # Create the DataFrame
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': feature_importances
        })

        # Sort the DataFrame by importance in descending order
        feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

        # 5. Print the feature importances
        print("\nFeature Importances from Random Forest Regressor:")
        display(feature_importance_df)

    else:
        print("X_train not available. Cannot determine feature names for importance analysis.")

else:
    print("Trained model or test data (X_test) not available for making predictions or gaining insights.")


Feature Importances from Random Forest Regressor:


,feature,importance
0,lat,0.506560
1,lon,0.454037
2,timestamp_hour,0.039403


## Visualize ml results

### Subtask:
Visualize the predictions or insights from the ML model, potentially overlaying them on the balloon trajectories or maps.

**Reasoning**:
Combine test features, actual, and predicted wind values into a single DataFrame and prepare for visualization. Create visualizations (e.g., scatter plots on a map, actual vs. predicted scatter plots) to show the model's performance and insights.

In [ ]:
import pandas as pd
import plotly.express as px

# Ensure test data and predictions are available
if 'X_test' in locals() and not X_test.empty and 'y_test' in locals() and not y_test.empty and 'y_pred' in locals():

    # 1. Create a new DataFrame combining the test features (X_test), the actual wind speed and direction (y_test), and the predicted wind speed and direction (y_pred).
    # Ensure the indices align correctly. X_test and y_test already have aligned indices.
    # y_pred is a numpy array, so we need to convert it to a DataFrame with matching indices.

    # Create DataFrames for predictions with the same index as X_test/y_test
    y_pred_df = pd.DataFrame(y_pred, index=X_test.index, columns=['predicted_wind_speed', 'predicted_wind_direction'])

    # Concatenate X_test, y_test, and the predicted values DataFrames
    # Use axis=1 to concatenate columns
    visualization_df = pd.concat([X_test, y_test, y_pred_df], axis=1)

    # 2. Rename the columns in the combined DataFrame for clarity (already done in y_pred_df columns)
    # Ensure column names are clear
    visualization_df.rename(columns={
        'wind_speed': 'actual_wind_speed',
        'wind_direction': 'actual_wind_direction'
    }, inplace=True)

    # 3. Create a scatter plot on a map using plotly.express to visualize the test locations.
    # 4. Color the points based on the difference between predicted and actual wind speed or wind direction to highlight areas where the model performs well or poorly.
    # Calculate the difference in wind speed and wind direction
    visualization_df['speed_difference'] = visualization_df['predicted_wind_speed'] - visualization_df['actual_wind_speed']
    visualization_df['direction_difference'] = visualization_df['predicted_wind_direction'] - visualization_df['actual_wind_direction']

    # Visualize using speed difference for coloring
    fig_speed_diff = px.scatter_mapbox(visualization_df,
                                       lat="lat",
                                       lon="lon",
                                       color="speed_difference", # Color by wind speed difference
                                       size="actual_wind_speed", # Size by actual wind speed (optional, or could size by magnitude of difference)
                                       hover_name=visualization_df.index, # Use index for hover name, which corresponds to the original row
                                       hover_data={
                                           "lat": True,
                                           "lon": True,
                                           "timestamp_hour": True,
                                           "actual_wind_speed": ':.2f',
                                           "predicted_wind_speed": ':.2f',
                                           "speed_difference": ':.2f',
                                           "actual_wind_direction": True,
                                           "predicted_wind_direction": True,
                                           "direction_difference": ':.2f'
                                           },
                                       color_continuous_scale="RdBu", # Red-Blue colormap for divergence (difference)
                                       size_max=10, # Adjust size based on data distribution
                                       zoom=1,
                                       height=600)

    # 6. Configure the map style and layout for better visualization.
    fig_speed_diff.update_layout(mapbox_style="open-street-map")
    fig_speed_diff.update_layout(margin={"r":0,"t":40,"l":0,"b":0}) # Add some top margin for the title
    fig_speed_diff.update_layout(title_text="Wind Speed Prediction Difference (Predicted - Actual) on Test Data") # Add a title

    # 7. Display the plot.
    fig_speed_diff.show()

    # Optional: Visualize using direction difference for coloring
    # Note: Wind direction is circular, simple difference might not be ideal for visualization.
    # A more advanced approach would handle the 0/360 degree wrap-around.
    # For this visualization, we'll use the simple difference but acknowledge its limitation for direction.
    fig_direction_diff = px.scatter_mapbox(visualization_df,
                                       lat="lat",
                                       lon="lon",
                                       color="direction_difference", # Color by wind direction difference
                                       size="actual_wind_direction", # Size by actual wind direction (less intuitive than speed)
                                       hover_name=visualization_df.index, # Use index for hover name
                                       hover_data={
                                           "lat": True,
                                           "lon": True,
                                           "timestamp_hour": True,
                                           "actual_wind_speed": ':.2f',
                                           "predicted_wind_speed": ':.2f',
                                           "speed_difference": ':.2f',
                                           "actual_wind_direction": True,
                                           "predicted_wind_direction": True,
                                           "direction_difference": ':.2f'
                                           },
                                       color_continuous_scale="RdBu",
                                       size_max=10,
                                       zoom=1,
                                       height=600)

    fig_direction_diff.update_layout(mapbox_style="open-street-map")
    fig_direction_diff.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig_direction_diff.update_layout(title_text="Wind Direction Prediction Difference (Predicted - Actual) on Test Data")
    fig_direction_diff.show()

    # 8. Optionally, create additional visualizations (e.g., scatter plots comparing actual vs. predicted values)
    fig_speed_scatter = px.scatter(visualization_df,
                                   x="actual_wind_speed",
                                   y="predicted_wind_speed",
                                   hover_data={"lat": True, "lon": True, "timestamp_hour": True, "speed_difference": True},
                                   title="Actual vs. Predicted Wind Speed")
    fig_speed_scatter.show()

    fig_direction_scatter = px.scatter(visualization_df,
                                      x="actual_wind_direction",
                                      y="predicted_wind_direction",
                                      hover_data={"lat": True, "lon": True, "timestamp_hour": True, "direction_difference": True},
                                      title="Actual vs. Predicted Wind Direction")
    fig_direction_scatter.show()

else:
    print("Test data (X_test, y_test) or predictions (y_pred) not available for visualization.")

In [55]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
import os
import time
import pickle
import requests # Import requests for data acquisition
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # Import the model

# --- Helper Functions (from the notebook) ---

def find_list_of_candidates(data):
    """Recursively searches for a list that potentially contains balloon data (dicts or lists)."""
    if isinstance(data, list):
        if any(isinstance(item, (dict, list)) for item in data):
             return data
        elif len(data) > 1 and all(isinstance(item, (int, float)) for item in data):
             return data
        else:
            for item in data:
                found = find_list_of_candidates(item)
                if found:
                    return found

    elif isinstance(data, dict):
        for key in ['balloons', 'data', 'positions', 'flights']:
            if key in data and isinstance(data[key], list):
                 if any(isinstance(item, (dict, list)) for item in data[key]):
                    return data[key]
        for value in data.values():
            found = find_list_of_candidates(value)
            if found:
                return found
    return None


def extract_balloon_data(raw_data, hour_index):
    extracted = []
    candidates = find_list_of_candidates(raw_data)

    if not candidates:
        # print(f"Could not find a list of candidates in data from hour {hour_index}. Raw data type: {type(raw_data)}") # Suppress verbose logging
        return extracted

    for i, entry in enumerate(candidates):
        try:
            lat = None
            lon = None
            balloon_id = None
            timestamp = f"{hour_index:02d}H_ago" # Default timestamp

            if isinstance(entry, dict):
                lat = entry.get('lat') or entry.get('latitude')
                lon = entry.get('lon') or entry.get('lng') or entry.get('longitude')
                balloon_id = entry.get('id') or entry.get('balloon_id') or entry.get('name')
                timestamp = entry.get('timestamp') or entry.get('time') or entry.get('ts')

            elif isinstance(entry, list) and len(entry) >= 2:
                lat = entry[0]
                lon = entry[1]
                balloon_id = f"balloon_{hour_index:02d}_{i}"

            if lat is not None and lon is not None:
                try:
                    lat = float(lat)
                    lon = float(lon)
                except (ValueError, TypeError):
                    # print(f"Skipping entry at index {i} from hour {hour_index} due to non-numeric lat/lon: lat={lat}, lon={lon}") # Suppress verbose logging
                    continue

                final_timestamp = timestamp if isinstance(entry, dict) and (entry.get('timestamp') or entry.get('time') or entry.get('ts')) else f"{hour_index:02d}H_ago"

                extracted.append({
                    'id': balloon_id,
                    'lat': lat,
                    'lon': lon,
                    'timestamp': final_timestamp,
                    'raw': entry
                })
            else:
                if isinstance(entry, (dict, list)):
                     pass # Suppress verbose logging
                else:
                     pass # Suppress verbose logging


        except (TypeError, AttributeError, IndexError, Exception) as e:
            # print(f"Error processing entry at index {i} from hour {hour_index}: {entry}. Error: {e}") # Suppress verbose logging
            continue

    return extracted

def fetch_and_extract_all():
    all_balloon_data = []
    st.info("Fetching balloon data...")
    progress_bar = st.progress(0)
    for hour in range(24):
        url = f"https://a.windbornesystems.com/treasure/{hour:02d}.json"
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            try:
                data = response.json()
                extracted = extract_balloon_data(data, hour)
                all_balloon_data.extend(extracted)
            except json.JSONDecodeError as e:
                st.warning(f"Malformed JSON at {url}: {e}, skipping.")
            except Exception as e:
                 st.warning(f"Error processing JSON data from {url}: {e}")
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404:
                st.warning(f"File not found at {url}, skipping.")
            else:
                st.warning(f"HTTP error fetching {url}: {e}")
        except Exception as e:
            st.warning(f"Error fetching {url}: {e}")
        progress_bar.progress((hour + 1) / 24)
    st.success(f"Fetched {len(all_balloon_data)} total balloon position entries.")
    return all_balloon_data

def get_weather_data(lat, lon, api_key):
    """
    Fetches wind speed and direction from OpenWeatherMap API for a given location.

    Args:
        lat (float): Latitude of the location.
        lon (float) : Longitude of the location.
        api_key (str): OpenWeatherMap API key.

    Returns:
        tuple: A tuple containing (wind_speed, wind_direction) in m/s and degrees,
               or (None, None) if the API call is unsuccessful or data is missing.
    """
    if not api_key:
        st.error("OpenWeatherMap API key not provided.")
        return None, None

    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}lat={lat}&lon={lon}&appid={api_key}&units=metric"

    try:
        response = requests.get(complete_url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "wind" in data:
            wind_speed = data["wind"].get("speed")
            wind_direction = data["wind"].get("deg")
            return wind_speed, wind_direction
        else:
            # st.warning(f"Wind data not found for lat={lat}, lon={lon}") # Suppress verbose logging
            return None, None

    except requests.exceptions.HTTPError as e:
        # st.warning(f"HTTP error fetching weather data for lat={lat}, lon={lon}: {e}") # Suppress verbose logging
        pass
    except requests.exceptions.ConnectionError as e:
        # st.warning(f"Connection error fetching weather data for lat={lat}, lon={lon}: {e}") # Suppress verbose logging
        pass
    except requests.exceptions.Timeout as e:
        # st.warning(f"Timeout error fetching weather data for lat={lat}, lon={lon}: {e}") # Suppress verbose logging
        pass
    except requests.exceptions.RequestException as e:
        # st.warning(f"Error fetching weather data for lat={lat}, lon={lon}: {e}") # Suppress verbose logging
        pass
    except json.JSONDecodeError:
        # st.warning(f"Malformed JSON response for lat={lat}, lon={lon}") # Suppress verbose logging
        pass
    except Exception as e:
        # st.warning(f"An unexpected error occurred fetching weather data for lat={lat}, lon={lon}: {e}") # Suppress verbose logging
        pass

    return None, None


def fetch_weather_for_all_balloon_data(all_balloon_data, api_key, limit=None):
    """Fetches weather data for each balloon entry, with an optional limit."""
    combined_data = []
    weather_fetch_count = 0
    successful_fetches = 0
    API_CALL_DELAY = 0.1

    st.info("Fetching weather data for balloon positions...")
    progress_bar = st.progress(0)

    data_to_process = all_balloon_data[:limit] if limit is not None else all_balloon_data

    if data_to_process:
        for i, entry in enumerate(data_to_process):
            lat = entry.get('lat')
            lon = entry.get('lon')

            try:
                lat = float(lat) if lat is not None else None
                lon = float(lon) if lon is not None else None
            except (ValueError, TypeError):
                lat = None
                lon = None

            if lat is not None and lon is not None:
                wind_speed, wind_direction = get_weather_data(lat, lon, api_key)
                weather_fetch_count += 1

                if wind_speed is not None and wind_direction is not None:
                    entry['wind_speed'] = wind_speed
                    entry['wind_direction'] = wind_direction
                    successful_fetches += 1
                else:
                    entry['wind_speed'] = None
                    entry['wind_direction'] = None

                combined_data.append(entry)
                time.sleep(API_CALL_DELAY)
            else:
                combined_data.append(entry) # Include entry even if lat/lon is missing

            progress_bar.progress((i + 1) / len(data_to_process))

        st.success(f"Attempted to fetch weather data for {weather_fetch_count} entries.")
        st.success(f"Successfully fetched weather data for {successful_fetches} entries.")
        st.success(f"Combined data contains {len(combined_data)} entries.")
    else:
        st.warning("No balloon data available for weather fetching.")

    return combined_data


def prepare_data_for_ml(combined_data):
    """Prepares combined data into a DataFrame for ML."""
    if not combined_data:
        st.warning("No combined data available to prepare for ML.")
        return pd.DataFrame()

    df = pd.DataFrame(combined_data)

    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['lon'] = pd.to_numeric(df['lon'], errors='coerce')
    df['wind_speed'] = pd.to_numeric(df['wind_speed'], errors='coerce')
    df['wind_direction'] = pd.to_numeric(df['wind_direction'], errors='coerce')

    df.dropna(subset=['lat', 'lon', 'wind_speed', 'wind_direction'], inplace=True)

    if 'timestamp' in df.columns:
        df['timestamp_hour'] = df['timestamp'].str.extract(r'(\d+)H_ago').astype(float)
        df = df.sort_values(by=['id', 'timestamp_hour']) # Keep sorting for potential future use of diff features

        # Calculate movement features (not used for the current ML problem but kept for completeness)
        df['delta_lat'] = df.groupby('id')['lat'].diff()
        df['delta_lon'] = df.groupby('id')['lon'].diff()
        df['time_diff_hours'] = df.groupby('id')['timestamp_hour'].diff()
        df['displacement'] = (df['delta_lat']**2 + df['delta_lon']**2)**0.5
        df['speed_approx'] = df['displacement'] / df['time_diff_hours']
    else:
        df['timestamp_hour'] = None # Ensure timestamp_hour column exists

    df.dropna(subset=['lat', 'lon', 'timestamp_hour', 'wind_speed', 'wind_direction'], inplace=True) # Drop rows with NaNs in ML features/targets


    st.success(f"Prepared DataFrame for ML with {len(df)} entries after cleaning.")
    return df

def train_ml_model(df):
    """Trains the Random Forest Regressor model."""
    if df.empty:
        st.warning("No data available to train the ML model.")
        return None, pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    X = df[['lat', 'lon', 'timestamp_hour']]
    y = df[['wind_speed', 'wind_direction']]

    if X.empty or y.empty:
         st.warning("Features or target variables are empty after selection.")
         return None, pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    st.info("Training Random Forest Regressor model...")
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    st.success("Model training complete.")

    return model, X_test, y_test, X_train # Return X_train for feature names


# --- Streamlit App ---

st.title("Windborne Balloon and Weather Data Analysis")

st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["Home", "Data Acquisition & Preparation", "ML Model", "Visualizations", "Predict Wind"])

# --- Store state using Streamlit's session_state ---
if 'df_combined' not in st.session_state:
    st.session_state.df_combined = pd.DataFrame()
if 'ml_model' not in st.session_state:
    st.session_state.ml_model = None
if 'X_test' not in st.session_state:
    st.session_state.X_test = pd.DataFrame()
if 'y_test' not in st.session_state:
    st.session_state.y_test = pd.DataFrame()
if 'X_train' not in st.session_state:
     st.session_state.X_train = pd.DataFrame()
if 'data_last_fetched' not in st.session_state:
    st.session_state.data_last_fetched = 0 # Timestamp of last fetch


# --- Data Refresh Logic ---
DATA_REFRESH_INTERVAL_SECONDS = 24 * 60 * 60 # 24 hours

def refresh_data_if_needed(openweathermap_api_key, num_entries_to_process):
    """Checks if data needs refreshing and fetches/prepares if necessary."""
    current_time = time.time()
    if current_time - st.session_state.data_last_fetched > DATA_REFRESH_INTERVAL_SECONDS or st.session_state.df_combined.empty:
        st.info("Data is older than 24 hours or not loaded. Refreshing data...")
        all_balloon_data = fetch_and_extract_all()
        if all_balloon_data:
            limit = num_entries_to_process if num_entries_to_process > 0 else None
            combined_data_with_weather = fetch_weather_for_all_balloon_data(all_balloon_data, openweathermap_api_key, limit=limit)
            if combined_data_with_weather:
                st.session_state.df_combined = prepare_data_for_ml(combined_data_with_weather)
                st.session_state.data_last_fetched = current_time # Update timestamp
                st.success("Data refresh complete.")
            else:
                st.warning("Data refresh failed: No combined data with weather was generated.")
                st.session_state.data_last_fetched = 0 # Reset timestamp if refresh fails
        else:
            st.warning("Data refresh failed: No balloon data was acquired.")
            st.session_state.data_last_fetched = 0 # Reset timestamp if refresh fails
    else:
        st.info("Data is up to date.")


if page == "Home":
    st.header("Welcome!")
    st.write("""
    This application analyzes Windborne balloon data combined with OpenWeatherMap weather data.
    It performs data acquisition, cleaning, prepares data for Machine Learning, trains a model
    to predict wind speed and direction, and provides visualizations and a prediction interface.

    The data is automatically refreshed every 24 hours.
    """)
    st.write("Use the sidebar to navigate through the different sections.")

elif page == "Data Acquisition & Preparation":
    st.header("Data Acquisition and Preparation")

    st.write("This section fetches balloon data from the Windborne API and weather data from OpenWeatherMap, then combines and prepares it for analysis and ML.")

    # Get OpenWeatherMap API key
    st.subheader("OpenWeatherMap API Key")
    openweathermap_api_key = st.text_input("Enter your OpenWeatherMap API Key", type="password")
    st.info("You can get an API key from the OpenWeatherMap website. Add it to your Colab secrets with the name `OPENWEATHERMAP_API_KEY` if running in Colab, or use environment variables in other environments.")

    # Add an input for the number of entries to process
    num_entries_to_process = st.number_input("Number of balloon entries to process for weather data (set to 0 for all)", min_value=0, value=100, step=10)


    if st.button("Run Data Acquisition and Preparation"):
        if not openweathermap_api_key:
            st.error("Please provide your OpenWeatherMap API key.")
        else:
            # This button will force a refresh regardless of the timestamp
            st.info("Forcing data refresh...")
            all_balloon_data = fetch_and_extract_all()
            if all_balloon_data:
                limit = num_entries_to_process if num_entries_to_process > 0 else None
                combined_data_with_weather = fetch_weather_for_all_balloon_data(all_balloon_data, openweathermap_api_key, limit=limit)
                if combined_data_with_weather:
                    st.session_state.df_combined = prepare_data_for_ml(combined_data_with_weather)
                    st.session_state.data_last_fetched = time.time() # Update timestamp on successful fetch
                    st.success("Data acquisition and preparation complete.")
                    if not st.session_state.df_combined.empty:
                        st.subheader("Prepared Data Preview")
                        st.dataframe(st.session_state.df_combined.head())
                else:
                    st.warning("No combined data with weather was generated.")
            else:
                st.warning("No balloon data was acquired.")
    else:
        # Automatic refresh when the page is loaded if data is old or not present
        refresh_data_if_needed(openweathermap_api_key, num_entries_to_process)


elif page == "ML Model":
    st.header("Machine Learning Model Training and Evaluation")

    if st.button("Train ML Model"):
        if not st.session_state.df_combined.empty:
            model, X_test, y_test, X_train = train_ml_model(st.session_state.df_combined)
            if model:
                st.session_state.ml_model = model
                st.session_state.X_test = X_test
                st.session_state.y_test = y_test
                st.session_state.X_train = X_train # Store X_train for feature names
            else:
                st.warning("Model training failed.")
        else:
            st.warning("Please run the data acquisition and preparation step first.")

    if st.session_state.ml_model:
        st.subheader("Model Evaluation Results on Test Data:")
        if not st.session_state.X_test.empty and not st.session_state.y_test.empty:
            try:
                y_pred = st.session_state.ml_model.predict(st.session_state.X_test)

                y_pred_speed = y_pred[:, 0]
                y_pred_direction = y_pred[:, 1]

                y_test_speed = st.session_state.y_test['wind_speed']
                y_test_direction = st.session_state.y_test['wind_direction']

                mse_speed = mean_squared_error(y_test_speed, y_pred_speed)
                mse_direction = mean_squared_error(y_test_direction, y_pred_direction)
                r2_speed = r2_score(y_test_speed, y_pred_speed)
                r2_direction = r2_score(y_test_direction, y_pred_direction)

                st.write(f"**Wind Speed:**")
                st.write(f"  - Mean Squared Error (MSE): {mse_speed:.4f}")
                st.write(f"  - R-squared: {r2_speed:.4f}")
                st.write(f"**Wind Direction:**")
                st.write(f"  - Mean Squared Error (MSE): {mse_direction:.4f}")
                st.write(f"  - R-squared: {r2_direction:.4f}")

                if hasattr(st.session_state.ml_model, 'feature_importances_') and not st.session_state.X_train.empty:
                    st.subheader("Feature Importances")
                    feature_names = st.session_state.X_train.columns
                    feature_importance_df = pd.DataFrame({
                        'feature': feature_names,
                        'importance': st.session_state.ml_model.feature_importances_
                    })
                    feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
                    st.dataframe(feature_importance_df)
                else:
                     st.info("Feature importances not available.")

            except Exception as e:
                st.error(f"An error occurred during model evaluation: {e}")

        else:
            st.info("Test data not available for evaluation. Please train the model first.")
    else:
        st.info("ML model not trained yet.")


elif page == "Visualizations":
    st.header("Visualizations")

    st.subheader("Combined Data Visualization (Actual Wind)")
    if not st.session_state.df_combined.empty:
        st.subheader("Filter Data")
        with st.form("filter_form_viz"):
            col1, col2 = st.columns(2)
            with col1:
                min_lat_viz = st.text_input("Min Latitude", key="min_lat_viz")
                max_lat_viz = st.text_input("Max Latitude", key="max_lat_viz")
            with col2:
                min_lon_viz = st.text_input("Min Longitude", key="min_lon_viz")
                max_lon_viz = st.text_input("Max Longitude", key="max_lon_viz")
            hour_filter_viz = st.text_input("Hour (0-23)", key="hour_filter_viz")

            # Add the submit button to the form
            apply_filter_viz = st.form_submit_button("Apply Filter")
            clear_filter_viz = st.form_submit_button("Clear Filter")


        filtered_df_viz = st.session_state.df_combined.copy()

        if apply_filter_viz:
            try:
                if min_lat_viz:
                    filtered_df_viz = filtered_df_viz[filtered_df_viz['lat'] >= float(min_lat_viz)]
                if max_lat_viz:
                    filtered_df_viz = filtered_df_viz[filtered_df_viz['lat'] <= float(max_lat_viz)]
                if min_lon_viz:
                    filtered_df_viz = filtered_df_viz[filtered_df_viz['lon'] >= float(min_lon_viz)]
                if max_lon_viz:
                    filtered_df_viz = filtered_df_viz[filtered_df_viz['lon'] <= float(max_lon_viz)]
                if hour_filter_viz:
                     hour_float_viz = float(hour_filter_viz)
                     if 'timestamp_hour' in filtered_df_viz.columns and not filtered_df_viz['timestamp_hour'].isnull().all():
                          filtered_df_viz = filtered_df_viz[filtered_df_viz['timestamp_hour'] == hour_float_viz]
                     else:
                          filtered_df_viz = filtered_df_viz.head(0)

            except ValueError:
                st.error("Invalid filter input. Please provide numeric values.")
                filtered_df_viz = st.session_state.df_combined.head(0)
            except Exception as e:
                st.error(f"An error occurred during filtering: {e}")
                filtered_df_viz = st.session_state.df_combined.head(0)

        st.subheader(f"Balloon Positions ({len(filtered_df_viz)} entries) with Actual Wind Speed (Filtered)")
        if not filtered_df_viz.empty and 'lat' in filtered_df_viz.columns and 'lon' in filtered_df_viz.columns and 'wind_speed' in filtered_df_viz.columns:
            fig_base_data = px.scatter_mapbox(filtered_df_viz,
                                    lat="lat",
                                    lon="lon",
                                    color="wind_speed",
                                    size="wind_speed",
                                    hover_name="id",
                                    hover_data={"lat": True, "lon": True, "wind_speed": True, "wind_direction": True, "timestamp": True},
                                    color_continuous_scale=px.colors.cyclical.IceFire,
                                    size_max=15,
                                    zoom=1,
                                    height=500)

            fig_base_data.update_layout(mapbox_style="open-street-map")
            fig_base_data.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
            st.plotly_chart(fig_base_data, use_container_width=True)
        else:
             st.info("No combined data available or matching filter criteria for base visualization.")

    else:
        st.info("No combined data available. Please run the data acquisition and preparation step.")


    st.subheader("Machine Learning Results Visualizations (on Test Data)")
    if st.session_state.ml_model and not st.session_state.X_test.empty and not st.session_state.y_test.empty:
        try:
            y_pred_viz = st.session_state.ml_model.predict(st.session_state.X_test)
            y_pred_df_viz = pd.DataFrame(y_pred_viz, index=st.session_state.X_test.index, columns=['predicted_wind_speed', 'predicted_wind_direction'])
            visualization_df_ml = pd.concat([st.session_state.X_test, st.session_state.y_test, y_pred_df_viz], axis=1)
            visualization_df_ml.rename(columns={
                'wind_speed': 'actual_wind_speed',
                'wind_direction': 'actual_wind_direction'
            }, inplace=True)

            visualization_df_ml['speed_difference'] = visualization_df_ml['predicted_wind_speed'] - visualization_df_ml['actual_wind_speed']
            visualization_df_ml['direction_difference'] = visualization_df_ml['predicted_wind_direction'] - visualization_df_ml['actual_wind_direction']

            st.subheader("Wind Speed Prediction Difference (Predicted - Actual)")
            fig_speed_diff = px.scatter_mapbox(visualization_df_ml,
                                               lat="lat",
                                               lon="lon",
                                               color="speed_difference",
                                               size="actual_wind_speed",
                                               hover_name=visualization_df_ml.index,
                                               hover_data={"lat": True, "lon": True, "timestamp_hour": True, "actual_wind_speed": ':.2f', "predicted_wind_speed": ':.2f', "speed_difference": ':.2f', "actual_wind_direction": True, "predicted_wind_direction": True, "direction_difference": ':.2f'},
                                               color_continuous_scale="RdBu",
                                               size_max=10,
                                               zoom=1,
                                               height=600)
            fig_speed_diff.update_layout(mapbox_style="open-street-map")
            fig_speed_diff.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
            st.plotly_chart(fig_speed_diff, use_container_width=True)

            st.subheader("Wind Direction Prediction Difference (Predicted - Actual)")
            fig_direction_diff = px.scatter_mapbox(visualization_df_ml,
                                               lat="lat",
                                               lon="lon",
                                               color="direction_difference",
                                               size="actual_wind_direction",
                                               hover_name=visualization_df_ml.index,
                                               hover_data={"lat": True, "lon": True, "timestamp_hour": True, "actual_wind_speed": ':.2f', "predicted_wind_speed": ':.2f', "speed_difference": ':.2f', "actual_wind_direction": True, "predicted_wind_direction": True, "direction_difference": ':.2f'},
                                               color_continuous_scale="RdBu",
                                               size_max=10,
                                               zoom=1,
                                               height=600)
            fig_direction_diff.update_layout(mapbox_style="open-street-map")
            fig_direction_diff.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
            st.plotly_chart(fig_direction_diff, use_container_width=True)

            st.subheader("Actual vs. Predicted Wind Speed")
            fig_speed_scatter = px.scatter(visualization_df_ml,
                                           x="actual_wind_speed",
                                           y="predicted_wind_speed",
                                           hover_data={"lat": True, "lon": True, "timestamp_hour": True, "speed_difference": ':.2f'},
                                           title="Actual vs. Predicted Wind Speed on Test Data")
            st.plotly_chart(fig_speed_scatter, use_container_width=True)

            st.subheader("Actual vs. Predicted Wind Direction")
            fig_direction_scatter = px.scatter(visualization_df_ml,
                                              x="actual_wind_direction",
                                              y="predicted_wind_direction",
                                              hover_data={"lat": True, "lon": True, "timestamp_hour": True, "direction_difference": ':.2f'},
                                              title="Actual vs. Predicted Wind Direction on Test Data")
            st.plotly_chart(fig_direction_scatter, use_container_width=True)

        except Exception as e:
            st.error(f"Error generating ML results visualizations: {e}")

    else:
        st.info("ML test data or predictions not available for visualization. Please train the model first.")


elif page == "Predict Wind":
    st.header("Get Wind Prediction")
    st.write("Enter location and hour to get wind speed and direction prediction from the trained model.")

    if st.session_state.ml_model:
        lat_input = st.text_input("Latitude", value="0.0", key="predict_lat")
        lon_input = st.text_input("Longitude", value="0.0", key="predict_lon")
        hour_input = st.text_input("Hour (0-23)", value="0", key="predict_hour")

        if st.button("Predict Wind", key="predict_button"):
            try:
                lat = float(lat_input)
                lon = float(lon_input)
                hour = float(hour_input)

                if not (0 <= hour <= 23):
                    st.error("Invalid hour value. Please provide a number between 0 and 23.")
                else:
                    input_data = pd.DataFrame([[lat, lon, hour]], columns=['lat', 'lon', 'timestamp_hour'])
                    prediction = st.session_state.ml_model.predict(input_data)
                    predicted_speed = prediction[0, 0]
                    predicted_direction = prediction[0, 1]

                    st.subheader("Prediction Result:")
                    st.write(f"Predicted Wind Speed: {predicted_speed:.2f} m/s")
                    st.write(f"Predicted Wind Direction: {predicted_direction:.2f} degrees")

            except ValueError:
                st.error("Invalid input. Please provide numeric values for latitude, longitude, and hour.")
            except Exception as e:
                st.error(f"An error occurred during prediction: {e}")
    else:
        st.info("ML model not trained yet. Please go to the 'ML Model' page to train the model.")

Overwriting streamlit_app.py


In [ ]:
if 'df_combined' in locals() and not df_combined.empty:
    # Select the relevant columns and display the first 100 entries
    display(df_combined[['lat', 'lon', 'timestamp']].head(100))
else:
    print("The df_combined DataFrame is not available or is empty. Please run the data acquisition and preparation steps first.")

,lat,lon,timestamp
0,70.789291,37.272190,00H_ago
1,47.252662,-62.017661,00H_ago
2,-17.851459,-46.920065,00H_ago
3,-8.862796,142.935869,00H_ago
4,-16.140942,74.202958,00H_ago
...,...,...,...
95,76.722523,-140.209429,00H_ago
96,-21.230315,-154.733739,00H_ago
97,-26.767380,165.249536,00H_ago
98,0.791221,75.687099,00H_ago
